## **Federated Learning for attack detection: 3 nodes sharing weights**

IDs from this file = **ID9xy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Transform to One Hot Encoding the Categories - normal, attack
    data_labels.insert(0, 'normal', data['attack_cat'].replace('normal', 1).replace(['dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(1, 'attack', data['attack_cat'].replace(['dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 1).replace('normal', 0))

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

### Experiments with datasets

#### Dataset 3A

In [105]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/797642493.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/797642493.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/797642493.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/797642493.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_mem

In [106]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [107]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id900.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [108]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [109]:
global_model = build_model((24,1,1))

In [110]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [111]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id900.hdf5')

Epoch 1/5
54/54 [==============================] - 11s 209ms/step - loss: 0.2622 - accuracy: 0.9723 - val_loss: 2.4833 - val_accuracy: 0.4716
Epoch 2/5
54/54 [==============================] - 11s 205ms/step - loss: 0.0308 - accuracy: 0.9925 - val_loss: 0.8942 - val_accuracy: 0.4838
Epoch 3/5
54/54 [==============================] - 11s 197ms/step - loss: 0.0249 - accuracy: 0.9925 - val_loss: 0.7772 - val_accuracy: 0.4957
Epoch 4/5
54/54 [==============================] - 10s 194ms/step - loss: 0.0232 - accuracy: 0.9925 - val_loss: 0.7315 - val_accuracy: 0.5003
Epoch 5/5
54/54 [==============================] - 11s 195ms/step - loss: 0.0213 - accuracy: 0.9930 - val_loss: 0.8568 - val_accuracy: 0.4869
Epoch 1/5
53/53 [==============================] - 10s 195ms/step - loss: 0.0192 - accuracy: 0.9936 - val_loss: 0.1865 - val_accuracy: 0.8994
Epoch 2/5
53/53 [==============================] - 11s 198ms/step - loss: 0.0167 - accuracy: 0.9946 - val_loss: 0.1187 - val_accuracy: 0.9317
Epoch 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 0.3903 - accuracy: 0.8656
Epoch 1/5
54/54 [==============================] - 11s 207ms/step - loss: 0.0161 - accuracy: 0.9952 - val_loss: 0.5910 - val_accuracy: 0.6879
Epoch 2/5
54/54 [==============================] - 11s 201ms/step - loss: 0.0151 - accuracy: 0.9955 - val_loss: 0.7827 - val_accuracy: 0.6541
Epoch 3/5
54/54 [==============================] - 11s 196ms/step - loss: 0.0147 - accuracy: 0.9959 - val_loss: 0.9354 - val_accuracy: 0.6345
Epoch 4/5
54/54 [==============================] - 11s 202ms/step - loss: 0.0147 - accuracy: 0.9958 - val_loss: 0.8145 - val_accuracy: 0.6640
Epoch 5/5
54/54 [==============================] - 10s 193ms/step - loss: 0.0144 - accuracy: 0.9959 - val_loss: 0.6760 - val_accuracy: 0.6904
Epoch 1/5
53/53 [==============================] - 10s 197ms/step - loss: 0.0138 - accuracy: 0.9960 - val_loss: 0.1039 - val_accuracy: 0.9537
Epoch 2/5
53/53 [==============================] - 10s 19

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 0.5744 - accuracy: 0.8114
Epoch 1/5
54/54 [==============================] - 11s 207ms/step - loss: 0.0137 - accuracy: 0.9961 - val_loss: 1.1457 - val_accuracy: 0.6317
Epoch 2/5
54/54 [==============================] - 11s 201ms/step - loss: 0.0131 - accuracy: 0.9962 - val_loss: 1.0356 - val_accuracy: 0.6490
Epoch 3/5
54/54 [==============================] - 11s 204ms/step - loss: 0.0130 - accuracy: 0.9962 - val_loss: 0.8606 - val_accuracy: 0.6917
Epoch 4/5
54/54 [==============================] - 11s 199ms/step - loss: 0.0127 - accuracy: 0.9963 - val_loss: 0.9322 - val_accuracy: 0.6846
Epoch 5/5
54/54 [==============================] - 11s 197ms/step - loss: 0.0130 - accuracy: 0.9964 - val_loss: 0.8153 - val_accuracy: 0.7060
Epoch 1/5
53/53 [==============================] - 11s 204ms/step - loss: 0.0117 - accuracy: 0.9964 - val_loss: 0.1511 - val_accuracy: 0.9406
Epoch 2/5
53/53 [==============================] - 11s 20

...

1721/1721 [==============================] - 9s 5ms/step - loss: 0.9974 - accuracy: 0.6988
Epoch 1/5
54/54 [==============================] - 11s 213ms/step - loss: 0.0115 - accuracy: 0.9967 - val_loss: 0.9178 - val_accuracy: 0.6906
Epoch 2/5
54/54 [==============================] - 11s 205ms/step - loss: 0.0107 - accuracy: 0.9969 - val_loss: 1.2270 - val_accuracy: 0.6236
Epoch 3/5
54/54 [==============================] - 11s 205ms/step - loss: 0.0106 - accuracy: 0.9970 - val_loss: 1.1372 - val_accuracy: 0.6602
Epoch 4/5
54/54 [==============================] - 11s 202ms/step - loss: 0.0101 - accuracy: 0.9969 - val_loss: 1.4994 - val_accuracy: 0.5900
Epoch 5/5
54/54 [==============================] - 11s 206ms/step - loss: 0.0101 - accuracy: 0.9971 - val_loss: 0.9682 - val_accuracy: 0.6717
Epoch 1/5
53/53 [==============================] - 11s 205ms/step - loss: 0.0102 - accuracy: 0.9968 - val_loss: 0.1522 - val_accuracy: 0.9414
Epoch 2/5
53/53 [==============================] - 11s 20

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 1.5897 - accuracy: 0.6106
Epoch 1/5
54/54 [==============================] - 12s 213ms/step - loss: 0.0095 - accuracy: 0.9971 - val_loss: 1.2756 - val_accuracy: 0.6308
Epoch 2/5
54/54 [==============================] - 11s 204ms/step - loss: 0.0091 - accuracy: 0.9971 - val_loss: 1.3870 - val_accuracy: 0.6141
Epoch 3/5
54/54 [==============================] - 11s 204ms/step - loss: 0.0090 - accuracy: 0.9973 - val_loss: 0.9764 - val_accuracy: 0.6837
Epoch 4/5
54/54 [==============================] - 11s 203ms/step - loss: 0.0089 - accuracy: 0.9974 - val_loss: 0.9309 - val_accuracy: 0.6724
Epoch 5/5
54/54 [==============================] - 11s 200ms/step - loss: 0.0091 - accuracy: 0.9971 - val_loss: 1.0666 - val_accuracy: 0.6524
Epoch 1/5
53/53 [==============================] - 11s 203ms/step - loss: 0.0088 - accuracy: 0.9970 - val_loss: 0.1297 - val_accuracy: 0.9511
Epoch 2/5
53/53 [==============================] - 11s 20

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 1.6865 - accuracy: 0.6054


In [112]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.15998446941375732, 0.3903036117553711], [0.1964522898197174, 0.574399471282959], [0.31562283635139465, 0.9973721504211426], [0.48944318294525146, 1.5897085666656494], [0.4267035126686096, 1.6865005493164062]]
Accuracy for iterations:  [[0.9844649434089661, 0.8655634522438049], [0.9818356037139893, 0.8113783597946167], [0.8254331350326538, 0.698793888092041], [0.7497735619544983, 0.610640823841095], [0.7546378970146179, 0.6053730845451355]]
F1 for iterations:  [[0.9844646174516402, 0.8614211411101467], [0.9818309522889497, 0.8006623300330845], [0.8187625365341832, 0.6546986901104187], [0.7294983754007442, 0.5084641884715652], [0.7356653483572342, 0.49782254466752396]]
Precision for iterations:  [[0.9844645402839748, 0.885309870993468], [0.9818596690859052, 0.8500236450380885], [0.862681096824321, 0.7921950741887742], [0.8245601765474275, 0.7530484567395882], [0.8262189548215215, 0.7540072721991109]]
Recall for iterations:  [[0.9844649274007421, 0.86556346726731

#### Dataset 3B 

In [113]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1167559547.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1167559547.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part2.csv')


In [114]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [115]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id901.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [116]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [117]:
global_model = build_model((24,1,1))

In [118]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [119]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id901.hdf5')

Epoch 1/5
57/57 [==============================] - 12s 209ms/step - loss: 0.1091 - accuracy: 0.9901 - val_loss: 1.8670e-05 - val_accuracy: 1.0000
Epoch 2/5
57/57 [==============================] - 12s 204ms/step - loss: 1.4703e-05 - accuracy: 1.0000 - val_loss: 1.3158e-05 - val_accuracy: 1.0000
Epoch 3/5
57/57 [==============================] - 12s 204ms/step - loss: 1.2687e-05 - accuracy: 1.0000 - val_loss: 1.1864e-05 - val_accuracy: 1.0000
Epoch 4/5
57/57 [==============================] - 12s 211ms/step - loss: 1.1381e-05 - accuracy: 1.0000 - val_loss: 1.0580e-05 - val_accuracy: 1.0000
Epoch 5/5
57/57 [==============================] - 13s 227ms/step - loss: 1.0109e-05 - accuracy: 1.0000 - val_loss: 9.3619e-06 - val_accuracy: 1.0000
Epoch 1/5
53/53 [==============================] - 11s 206ms/step - loss: 0.9348 - accuracy: 0.6703 - val_loss: 0.8597 - val_accuracy: 0.0000e+00
Epoch 2/5
53/53 [==============================] - 11s 202ms/step - loss: 0.2810 - accuracy: 0.9095 - val_lo

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 6s 4ms/step - loss: 0.5905 - accuracy: 0.5657
Epoch 1/5
57/57 [==============================] - 12s 203ms/step - loss: 0.0038 - accuracy: 0.9998 - val_loss: 1.7362e-06 - val_accuracy: 1.0000
Epoch 2/5
57/57 [==============================] - 11s 201ms/step - loss: 1.6415e-06 - accuracy: 1.0000 - val_loss: 1.5622e-06 - val_accuracy: 1.0000
Epoch 3/5
57/57 [==============================] - 12s 202ms/step - loss: 1.5494e-06 - accuracy: 1.0000 - val_loss: 1.4849e-06 - val_accuracy: 1.0000
Epoch 4/5
57/57 [==============================] - 12s 203ms/step - loss: 1.4672e-06 - accuracy: 1.0000 - val_loss: 1.4011e-06 - val_accuracy: 1.0000
Epoch 5/5
57/57 [==============================] - 11s 201ms/step - loss: 1.3805e-06 - accuracy: 1.0000 - val_loss: 1.3136e-06 - val_accuracy: 1.0000
Epoch 1/5
53/53 [==============================] - 11s 205ms/step - loss: 0.3459 - accuracy: 0.9312 - val_loss: 0.4207 - val_accuracy: 0.8896
Epoch 2/5
53/53 [====

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 0.3406 - accuracy: 0.8436
Epoch 1/5
57/57 [==============================] - 12s 209ms/step - loss: 0.0218 - accuracy: 0.9921 - val_loss: 2.4394e-05 - val_accuracy: 1.0000
Epoch 2/5
57/57 [==============================] - 12s 204ms/step - loss: 2.1208e-05 - accuracy: 1.0000 - val_loss: 1.9105e-05 - val_accuracy: 1.0000
Epoch 3/5
57/57 [==============================] - 11s 201ms/step - loss: 1.7509e-05 - accuracy: 1.0000 - val_loss: 1.5890e-05 - val_accuracy: 1.0000
Epoch 4/5
57/57 [==============================] - 12s 203ms/step - loss: 1.4517e-05 - accuracy: 1.0000 - val_loss: 1.3106e-05 - val_accuracy: 1.0000
Epoch 5/5
57/57 [==============================] - 12s 204ms/step - loss: 1.2004e-05 - accuracy: 1.0000 - val_loss: 1.0835e-05 - val_accuracy: 1.0000
Epoch 1/5
53/53 [==============================] - 11s 203ms/step - loss: 0.1508 - accuracy: 0.9583 - val_loss: 0.2777 - val_accuracy: 0.8897
Epoch 2/5
53/53 [====

...

1721/1721 [==============================] - 9s 5ms/step - loss: 0.5751 - accuracy: 0.7483
Epoch 1/5
57/57 [==============================] - 12s 205ms/step - loss: 0.0359 - accuracy: 0.9853 - val_loss: 1.2261e-04 - val_accuracy: 1.0000
Epoch 2/5
57/57 [==============================] - 12s 205ms/step - loss: 8.3455e-05 - accuracy: 1.0000 - val_loss: 5.7423e-05 - val_accuracy: 1.0000
Epoch 3/5
57/57 [==============================] - 11s 198ms/step - loss: 4.5226e-05 - accuracy: 1.0000 - val_loss: 3.5287e-05 - val_accuracy: 1.0000
Epoch 4/5
57/57 [==============================] - 11s 198ms/step - loss: 2.9212e-05 - accuracy: 1.0000 - val_loss: 2.3806e-05 - val_accuracy: 1.0000
Epoch 5/5
57/57 [==============================] - 11s 195ms/step - loss: 2.0453e-05 - accuracy: 1.0000 - val_loss: 1.7365e-05 - val_accuracy: 1.0000
Epoch 1/5
53/53 [==============================] - 11s 199ms/step - loss: 0.1201 - accuracy: 0.9627 - val_loss: 0.2220 - val_accuracy: 0.8897
Epoch 2/5
53/53 [====

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 0.7857 - accuracy: 0.6959
Epoch 1/5
57/57 [==============================] - 12s 212ms/step - loss: 0.0449 - accuracy: 0.9810 - val_loss: 1.2543e-04 - val_accuracy: 1.0000
Epoch 2/5
57/57 [==============================] - 12s 206ms/step - loss: 9.0821e-05 - accuracy: 1.0000 - val_loss: 7.0171e-05 - val_accuracy: 1.0000
Epoch 3/5
57/57 [==============================] - 11s 196ms/step - loss: 5.8226e-05 - accuracy: 1.0000 - val_loss: 4.8011e-05 - val_accuracy: 1.0000
Epoch 4/5
57/57 [==============================] - 11s 200ms/step - loss: 4.0903e-05 - accuracy: 1.0000 - val_loss: 3.4981e-05 - val_accuracy: 1.0000
Epoch 5/5
57/57 [==============================] - 11s 196ms/step - loss: 3.0257e-05 - accuracy: 1.0000 - val_loss: 2.6414e-05 - val_accuracy: 1.0000
Epoch 1/5
53/53 [==============================] - 11s 200ms/step - loss: 0.0945 - accuracy: 0.9685 - val_loss: 0.1829 - val_accuracy: 0.8940
Epoch 2/5
53/53 [====

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 0.8097 - accuracy: 0.6941


In [120]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.6053115725517273, 0.5905081629753113], [0.30678895115852356, 0.34060606360435486], [0.5845730304718018, 0.5750745534896851], [0.8395817875862122, 0.785650372505188], [0.8872372508049011, 0.8097327351570129]]
Accuracy for iterations:  [[0.5278930068016052, 0.5657378435134888], [0.8622586131095886, 0.8435660600662231], [0.7529653310775757, 0.7483470439910889], [0.6975517868995667, 0.6958693861961365], [0.6954994201660156, 0.6940710544586182]]
F1 for iterations:  [[0.3654250118828627, 0.41558303250829143], [0.8607483495500788, 0.8428270912895665], [0.7409784957190669, 0.740043628729495], [0.6737323040995583, 0.6784019834683847], [0.6711269993979918, 0.6762043226949995]]
Precision for iterations:  [[0.28681731911139235, 0.69834494909611], [0.8931040003051779, 0.882626725259436], [0.8378945143458204, 0.8393559359063927], [0.8157405920823093, 0.8202588389389841], [0.8149807420106494, 0.819670241181861]]
Recall for iterations:  [[0.5278930146951415, 0.565737847852939

#### Dataset 3C

In [64]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low

In [65]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [66]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id902.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [67]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [68]:
global_model = build_model((24,1,1))

In [69]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [70]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id902.hdf5')

Epoch 1/15
54/54 [==============================] - 12s 219ms/step - loss: 0.2487 - accuracy: 0.9682 - val_loss: 1.5603 - val_accuracy: 0.6400
Epoch 2/15
54/54 [==============================] - 11s 211ms/step - loss: 0.0327 - accuracy: 0.9921 - val_loss: 0.5386 - val_accuracy: 0.6829
Epoch 3/15
54/54 [==============================] - 12s 213ms/step - loss: 0.0252 - accuracy: 0.9922 - val_loss: 0.4977 - val_accuracy: 0.6937
Epoch 4/15
54/54 [==============================] - 12s 217ms/step - loss: 0.0235 - accuracy: 0.9923 - val_loss: 0.4973 - val_accuracy: 0.6806
Epoch 5/15
54/54 [==============================] - 11s 207ms/step - loss: 0.0216 - accuracy: 0.9927 - val_loss: 0.5465 - val_accuracy: 0.6619
Epoch 6/15
54/54 [==============================] - 10s 184ms/step - loss: 0.0201 - accuracy: 0.9934 - val_loss: 0.5263 - val_accuracy: 0.6795
Epoch 7/15
54/54 [==============================] - 10s 187ms/step - loss: 0.0184 - accuracy: 0.9940 - val_loss: 0.3709 - val_accuracy: 0.7692

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 1.1312 - accuracy: 0.7386
Epoch 1/15
54/54 [==============================] - 11s 196ms/step - loss: 0.0123 - accuracy: 0.9963 - val_loss: 0.4582 - val_accuracy: 0.8213
Epoch 2/15
54/54 [==============================] - 10s 192ms/step - loss: 0.0116 - accuracy: 0.9967 - val_loss: 0.5356 - val_accuracy: 0.8009
Epoch 3/15
54/54 [==============================] - 10s 191ms/step - loss: 0.0113 - accuracy: 0.9966 - val_loss: 0.6794 - val_accuracy: 0.7605
Epoch 4/15
54/54 [==============================] - 10s 191ms/step - loss: 0.0110 - accuracy: 0.9967 - val_loss: 0.4375 - val_accuracy: 0.8341
Epoch 5/15
54/54 [==============================] - 10s 188ms/step - loss: 0.0105 - accuracy: 0.9970 - val_loss: 0.6766 - val_accuracy: 0.7687
Epoch 6/15
54/54 [==============================] - 10s 190ms/step - loss: 0.0107 - accuracy: 0.9969 - val_loss: 0.3380 - val_accuracy: 0.8551
Epoch 7/15
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 6s 4ms/step - loss: 2.3766 - accuracy: 0.6404
Epoch 1/15
54/54 [==============================] - 10s 190ms/step - loss: 0.0078 - accuracy: 0.9978 - val_loss: 0.8014 - val_accuracy: 0.7748
Epoch 2/15
54/54 [==============================] - 10s 188ms/step - loss: 0.0073 - accuracy: 0.9979 - val_loss: 0.8290 - val_accuracy: 0.7617
Epoch 3/15
54/54 [==============================] - 10s 189ms/step - loss: 0.0075 - accuracy: 0.9979 - val_loss: 0.4031 - val_accuracy: 0.8510
Epoch 4/15
54/54 [==============================] - 10s 189ms/step - loss: 0.0071 - accuracy: 0.9979 - val_loss: 0.4746 - val_accuracy: 0.8376
Epoch 5/15
54/54 [==============================] - 10s 188ms/step - loss: 0.0075 - accuracy: 0.9980 - val_loss: 0.4255 - val_accuracy: 0.8458
Epoch 6/15
54/54 [==============================] - 10s 186ms/step - loss: 0.0075 - accuracy: 0.9976 - val_loss: 0.8259 - val_accuracy: 0.7676
Epoch 7/15
54/54 [==============================] -

...

1721/1721 [==============================] - 7s 4ms/step - loss: 4.0949 - accuracy: 0.8533
Epoch 1/15
54/54 [==============================] - 11s 201ms/step - loss: 0.0039 - accuracy: 0.9988 - val_loss: 0.6839 - val_accuracy: 0.8078
Epoch 2/15
54/54 [==============================] - 11s 204ms/step - loss: 0.0035 - accuracy: 0.9990 - val_loss: 0.4978 - val_accuracy: 0.8431
Epoch 3/15
54/54 [==============================] - 11s 206ms/step - loss: 0.0039 - accuracy: 0.9987 - val_loss: 0.7517 - val_accuracy: 0.8004
Epoch 4/15
54/54 [==============================] - 11s 199ms/step - loss: 0.0038 - accuracy: 0.9990 - val_loss: 0.7580 - val_accuracy: 0.7912
Epoch 5/15
54/54 [==============================] - 11s 198ms/step - loss: 0.0033 - accuracy: 0.9990 - val_loss: 0.6523 - val_accuracy: 0.8054
Epoch 6/15
54/54 [==============================] - 11s 201ms/step - loss: 0.0031 - accuracy: 0.9989 - val_loss: 0.5849 - val_accuracy: 0.8207
Epoch 7/15
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 4.3419 - accuracy: 0.8513
Epoch 1/15
54/54 [==============================] - 11s 202ms/step - loss: 0.0033 - accuracy: 0.9989 - val_loss: 0.7358 - val_accuracy: 0.7960
Epoch 2/15
54/54 [==============================] - 10s 192ms/step - loss: 0.0032 - accuracy: 0.9990 - val_loss: 0.6417 - val_accuracy: 0.8074
Epoch 3/15
54/54 [==============================] - 10s 193ms/step - loss: 0.0028 - accuracy: 0.9990 - val_loss: 0.6925 - val_accuracy: 0.8048
Epoch 4/15
54/54 [==============================] - 10s 187ms/step - loss: 0.0029 - accuracy: 0.9991 - val_loss: 0.8167 - val_accuracy: 0.7812
Epoch 5/15
54/54 [==============================] - 10s 188ms/step - loss: 0.0029 - accuracy: 0.9991 - val_loss: 0.6488 - val_accuracy: 0.8146
Epoch 6/15
54/54 [==============================] - 10s 186ms/step - loss: 0.0032 - accuracy: 0.9990 - val_loss: 0.5134 - val_accuracy: 0.8382
Epoch 7/15
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 4.0738 - accuracy: 0.8464


In [71]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.8894578814506531, 1.1312083005905151], [1.698499321937561, 2.376584053039551], [2.183187961578369, 2.635972261428833], [2.3790643215179443, 2.703113317489624], [2.959472179412842, 3.0835437774658203], [2.8903372287750244, 3.248662233352661], [2.7629005908966064, 3.6125941276550293], [3.05550217628479, 4.0948567390441895], [3.1349785327911377, 4.341891288757324], [2.069155693054199, 4.0738444328308105]]
Accuracy for iterations:  [[0.5954526662826538, 0.7386289238929749], [0.5880978107452393, 0.640449047088623], [0.5912238359451294, 0.6655343770980835], [0.5924143195152283, 0.7065320014953613], [0.5908367037773132, 0.7559398412704468], [0.5916693210601807, 0.8232035040855408], [0.5930789709091187, 0.8535566329956055], [0.5904204249382019, 0.8532660007476807], [0.5943644046783447, 0.8513224124908447], [0.5977387428283691, 0.8463997840881348]]
F1 for iterations:  [[0.5064955898733743, 0.7102483789718483], [0.491785461370985, 0.5619450851264524], [0.498179759490840

#### Dataset 3D

In [121]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3D-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3D-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3D-Part3.csv')

In [122]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [123]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id903.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [124]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [125]:
global_model = build_model((24,1,1))

In [126]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [127]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id903.hdf5')

Epoch 1/5
22/22 [==============================] - 5s 208ms/step - loss: 0.3290 - accuracy: 0.9749 - val_loss: 0.0105 - val_accuracy: 1.0000
Epoch 2/5
22/22 [==============================] - 4s 204ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 5.0321e-05 - val_accuracy: 1.0000
Epoch 3/5
22/22 [==============================] - 4s 192ms/step - loss: 2.8219e-05 - accuracy: 1.0000 - val_loss: 1.8875e-05 - val_accuracy: 1.0000
Epoch 4/5
22/22 [==============================] - 4s 193ms/step - loss: 1.7149e-05 - accuracy: 1.0000 - val_loss: 1.6246e-05 - val_accuracy: 1.0000
Epoch 5/5
22/22 [==============================] - 4s 193ms/step - loss: 1.5764e-05 - accuracy: 1.0000 - val_loss: 1.5501e-05 - val_accuracy: 1.0000
Epoch 1/5
22/22 [==============================] - 4s 201ms/step - loss: 4.1748 - accuracy: 0.0000e+00 - val_loss: 1.1156 - val_accuracy: 0.0000e+00
Epoch 2/5
22/22 [==============================] - 4s 193ms/step - loss: 0.7547 - accuracy: 0.5041 - val_loss: 0.5952 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


   1/1721 [..............................] - ETA: 0s - loss: 0.0085 - accuracy: 1.0000

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1721/1721 [==============================] - 10s 6ms/step - loss: 2.8321 - accuracy: 0.4392


...

Epoch 1/5
22/22 [==============================] - 5s 211ms/step - loss: 0.6573 - accuracy: 0.7585 - val_loss: 5.1269e-05 - val_accuracy: 1.0000
Epoch 2/5
22/22 [==============================] - 5s 205ms/step - loss: 1.4026e-05 - accuracy: 1.0000 - val_loss: 4.0885e-06 - val_accuracy: 1.0000
Epoch 3/5
22/22 [==============================] - 4s 199ms/step - loss: 3.2955e-06 - accuracy: 1.0000 - val_loss: 2.9363e-06 - val_accuracy: 1.0000
Epoch 4/5
22/22 [==============================] - 4s 200ms/step - loss: 2.8361e-06 - accuracy: 1.0000 - val_loss: 2.8198e-06 - val_accuracy: 1.0000
Epoch 5/5
22/22 [==============================] - 4s 198ms/step - loss: 2.7832e-06 - accuracy: 1.0000 - val_loss: 2.8007e-06 - val_accuracy: 1.0000
Epoch 1/5
22/22 [==============================] - 5s 206ms/step - loss: 2.4579 - accuracy: 0.5385 - val_loss: 0.0038 - val_accuracy: 1.0000
Epoch 2/5
22/22 [==============================] - 4s 196ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 5.1172e

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 2.5579 - accuracy: 0.4393
Epoch 1/5
22/22 [==============================] - 5s 206ms/step - loss: 0.5775 - accuracy: 0.7934 - val_loss: 1.2236e-05 - val_accuracy: 1.0000
Epoch 2/5
22/22 [==============================] - 5s 207ms/step - loss: 3.3915e-06 - accuracy: 1.0000 - val_loss: 1.0271e-06 - val_accuracy: 1.0000
Epoch 3/5
22/22 [==============================] - 4s 203ms/step - loss: 8.2872e-07 - accuracy: 1.0000 - val_loss: 7.4032e-07 - val_accuracy: 1.0000
Epoch 4/5
22/22 [==============================] - 4s 203ms/step - loss: 7.1496e-07 - accuracy: 1.0000 - val_loss: 7.1269e-07 - val_accuracy: 1.0000
Epoch 5/5
22/22 [==============================] - 4s 197ms/step - loss: 7.0278e-07 - accuracy: 1.0000 - val_loss: 7.0914e-07 - val_accuracy: 1.0000
Epoch 1/5
22/22 [==============================] - 4s 200ms/step - loss: 2.6213 - accuracy: 0.5405 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 2/5
22/22 [==========

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 2.8714 - accuracy: 0.4393


In [128]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[2.693828582763672, 2.8321120738983154], [2.3076114654541016, 2.4550936222076416], [2.325613498687744, 2.483102560043335], [2.385244131088257, 2.5578691959381104], [2.674684762954712, 2.8713796138763428]]
Accuracy for iterations:  [[0.4712086319923401, 0.4392392635345459], [0.4712086319923401, 0.4392392635345459], [0.4712086319923401, 0.4392392635345459], [0.4711940288543701, 0.4393119215965271], [0.47125244140625, 0.4393119215965271]]
F1 for iterations:  [[0.30184375474744357, 0.2681015400046588], [0.30184375474744357, 0.2681015400046588], [0.30184375474744357, 0.2681015400046588], [0.3018373944883582, 0.26826037385846335], [0.30200498464953024, 0.26826037385846335]]
Precision for iterations:  [[0.2220375675826312, 0.19293113164997755], [0.2220375675826312, 0.19293113164997755], [0.2220375675826312, 0.19293113164997755], [0.22203392776410738, 0.7537058860749408], [0.5855903787793216, 0.7537058860749408]]
Recall for iterations:  [[0.4712086242659733, 0.4392392646

#### DATASET 3E

In [129]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3E-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3E-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3E-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/4252152495.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3E-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/4252152495.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3E-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/4252152495.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3E-Part3.csv')


In [130]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [131]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id904.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [132]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [133]:
global_model = build_model((24,1,1))

In [134]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [135]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id904.hdf5')

Epoch 1/5
54/54 [==============================] - 11s 203ms/step - loss: 0.2464 - accuracy: 0.9746 - val_loss: 1.6468 - val_accuracy: 0.6841
Epoch 2/5
54/54 [==============================] - 11s 198ms/step - loss: 0.0307 - accuracy: 0.9925 - val_loss: 0.5407 - val_accuracy: 0.6943
Epoch 3/5
54/54 [==============================] - 11s 199ms/step - loss: 0.0247 - accuracy: 0.9927 - val_loss: 0.5423 - val_accuracy: 0.6920
Epoch 4/5
54/54 [==============================] - 11s 196ms/step - loss: 0.0231 - accuracy: 0.9927 - val_loss: 0.4365 - val_accuracy: 0.7194
Epoch 5/5
54/54 [==============================] - 11s 199ms/step - loss: 0.0215 - accuracy: 0.9930 - val_loss: 0.4015 - val_accuracy: 0.7443
Epoch 1/5
53/53 [==============================] - 11s 203ms/step - loss: 0.0197 - accuracy: 0.9933 - val_loss: 0.5437 - val_accuracy: 0.6690
Epoch 2/5
53/53 [==============================] - 11s 209ms/step - loss: 0.0170 - accuracy: 0.9943 - val_loss: 0.3101 - val_accuracy: 0.8086
Epoch 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 0.3877 - accuracy: 0.8716
Epoch 1/5
54/54 [==============================] - 11s 212ms/step - loss: 0.0156 - accuracy: 0.9956 - val_loss: 0.3976 - val_accuracy: 0.8023
Epoch 2/5
54/54 [==============================] - 11s 205ms/step - loss: 0.0149 - accuracy: 0.9955 - val_loss: 0.2155 - val_accuracy: 0.8959
Epoch 3/5
54/54 [==============================] - 11s 205ms/step - loss: 0.0146 - accuracy: 0.9958 - val_loss: 0.2083 - val_accuracy: 0.9001
Epoch 4/5
54/54 [==============================] - 11s 203ms/step - loss: 0.0146 - accuracy: 0.9960 - val_loss: 0.2701 - val_accuracy: 0.8767
Epoch 5/5
54/54 [==============================] - 11s 207ms/step - loss: 0.0150 - accuracy: 0.9959 - val_loss: 0.2931 - val_accuracy: 0.8670
Epoch 1/5
53/53 [==============================] - 11s 210ms/step - loss: 0.0133 - accuracy: 0.9961 - val_loss: 0.5178 - val_accuracy: 0.7846
Epoch 2/5
53/53 [==============================] - 11s 20

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 0.6498 - accuracy: 0.7793
Epoch 1/5
54/54 [==============================] - 12s 220ms/step - loss: 0.0135 - accuracy: 0.9962 - val_loss: 0.2849 - val_accuracy: 0.8813
Epoch 2/5
54/54 [==============================] - 11s 209ms/step - loss: 0.0126 - accuracy: 0.9965 - val_loss: 0.3949 - val_accuracy: 0.8322
Epoch 3/5
54/54 [==============================] - 11s 206ms/step - loss: 0.0127 - accuracy: 0.9965 - val_loss: 0.3896 - val_accuracy: 0.8334
Epoch 4/5
54/54 [==============================] - 11s 203ms/step - loss: 0.0120 - accuracy: 0.9964 - val_loss: 0.3986 - val_accuracy: 0.8337
Epoch 5/5
54/54 [==============================] - 11s 206ms/step - loss: 0.0115 - accuracy: 0.9967 - val_loss: 0.3674 - val_accuracy: 0.8460
Epoch 1/5
53/53 [==============================] - 11s 208ms/step - loss: 0.0114 - accuracy: 0.9964 - val_loss: 0.8417 - val_accuracy: 0.7272
Epoch 2/5
53/53 [==============================] - 11s 20

...

1721/1721 [==============================] - 11s 7ms/step - loss: 1.1174 - accuracy: 0.6718
Epoch 1/5
54/54 [==============================] - 12s 215ms/step - loss: 0.0111 - accuracy: 0.9967 - val_loss: 0.3181 - val_accuracy: 0.8681
Epoch 2/5
54/54 [==============================] - 11s 210ms/step - loss: 0.0103 - accuracy: 0.9969 - val_loss: 0.2596 - val_accuracy: 0.8966
Epoch 3/5
54/54 [==============================] - 11s 202ms/step - loss: 0.0100 - accuracy: 0.9969 - val_loss: 0.2489 - val_accuracy: 0.9020
Epoch 4/5
54/54 [==============================] - 11s 204ms/step - loss: 0.0102 - accuracy: 0.9971 - val_loss: 0.4179 - val_accuracy: 0.8374
Epoch 5/5
54/54 [==============================] - 11s 204ms/step - loss: 0.0096 - accuracy: 0.9970 - val_loss: 0.3344 - val_accuracy: 0.8749
Epoch 1/5
53/53 [==============================] - 11s 205ms/step - loss: 0.0092 - accuracy: 0.9968 - val_loss: 0.6049 - val_accuracy: 0.7917
Epoch 2/5
53/53 [==============================] - 11s 2

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 1.5547 - accuracy: 0.6180
Epoch 1/5
54/54 [==============================] - 12s 213ms/step - loss: 0.0090 - accuracy: 0.9973 - val_loss: 0.4033 - val_accuracy: 0.8498
Epoch 2/5
54/54 [==============================] - 11s 203ms/step - loss: 0.0092 - accuracy: 0.9974 - val_loss: 0.3681 - val_accuracy: 0.8668
Epoch 3/5
54/54 [==============================] - 11s 204ms/step - loss: 0.0083 - accuracy: 0.9975 - val_loss: 0.2280 - val_accuracy: 0.9125
Epoch 4/5
54/54 [==============================] - 11s 203ms/step - loss: 0.0087 - accuracy: 0.9975 - val_loss: 0.3635 - val_accuracy: 0.8675
Epoch 5/5
54/54 [==============================] - 11s 204ms/step - loss: 0.0079 - accuracy: 0.9978 - val_loss: 0.3895 - val_accuracy: 0.8699
Epoch 1/5
53/53 [==============================] - 11s 205ms/step - loss: 0.0085 - accuracy: 0.9971 - val_loss: 0.8961 - val_accuracy: 0.7376
Epoch 2/5
53/53 [==============================] - 11s 20

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 2.0700 - accuracy: 0.5899


In [136]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1806637942790985, 0.3877066373825073], [0.21619848906993866, 0.6498269438743591], [0.30425554513931274, 1.1174098253250122], [0.36150333285331726, 1.5547280311584473], [0.5809736251831055, 2.0700178146362305]]
Accuracy for iterations:  [[0.9847205877304077, 0.8716304302215576], [0.9722238183021545, 0.7793177366256714], [0.8484618067741394, 0.6717830300331116], [0.8182169795036316, 0.6180338859558105], [0.7716994285583496, 0.5899149775505066]]
F1 for iterations:  [[0.98472053278769, 0.8679929656747903], [0.9721922336581498, 0.762200595142955], [0.8438955992323816, 0.6138388033350743], [0.8102776043676677, 0.5219490828372381], [0.7562099328558912, 0.46691651454452276]]
Precision for iterations:  [[0.9847205086710894, 0.8894972289163653], [0.9726307778196812, 0.8323628994196145], [0.8782689757051837, 0.780017108857993], [0.8614185860310581, 0.7586976231090304], [0.8361581849813545, 0.7474853648839689]]
Recall for iterations:  [[0.9847205585906688, 0.8716304584756

#### DATASET 3F

In [137]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3F-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3F-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3F-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/3013475373.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3F-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/3013475373.py:3: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3F-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/3013475373.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3F-Part3.csv')


In [138]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [139]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id905.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [140]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [141]:
global_model = build_model((24,1,1))

In [142]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [143]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id905.hdf5')

Epoch 1/5
34/34 [==============================] - 7s 204ms/step - loss: 0.4040 - accuracy: 0.8599 - val_loss: 2.4603 - val_accuracy: 0.2388
Epoch 2/5
34/34 [==============================] - 7s 203ms/step - loss: 0.0929 - accuracy: 0.9766 - val_loss: 0.7102 - val_accuracy: 0.6295
Epoch 3/5
34/34 [==============================] - 7s 206ms/step - loss: 0.0469 - accuracy: 0.9805 - val_loss: 0.4784 - val_accuracy: 0.7804
Epoch 4/5
34/34 [==============================] - 7s 206ms/step - loss: 0.0404 - accuracy: 0.9837 - val_loss: 0.4347 - val_accuracy: 0.8299
Epoch 5/5
34/34 [==============================] - 7s 204ms/step - loss: 0.0364 - accuracy: 0.9860 - val_loss: 0.3766 - val_accuracy: 0.8829
Epoch 1/5
53/53 [==============================] - 11s 208ms/step - loss: 0.0214 - accuracy: 0.9929 - val_loss: 0.0172 - val_accuracy: 0.9915
Epoch 2/5
53/53 [==============================] - 11s 209ms/step - loss: 0.0186 - accuracy: 0.9939 - val_loss: 0.0217 - val_accuracy: 0.9903
Epoch 3/5
5

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 0.3157 - accuracy: 0.8861
Epoch 1/5
34/34 [==============================] - 7s 196ms/step - loss: 0.0265 - accuracy: 0.9887 - val_loss: 0.5792 - val_accuracy: 0.6547
Epoch 2/5
34/34 [==============================] - 7s 195ms/step - loss: 0.0246 - accuracy: 0.9896 - val_loss: 0.4855 - val_accuracy: 0.6951
Epoch 3/5
34/34 [==============================] - 7s 193ms/step - loss: 0.0235 - accuracy: 0.9904 - val_loss: 0.4095 - val_accuracy: 0.7415
Epoch 4/5
34/34 [==============================] - 7s 196ms/step - loss: 0.0229 - accuracy: 0.9907 - val_loss: 0.6000 - val_accuracy: 0.6038
Epoch 5/5
34/34 [==============================] - 7s 196ms/step - loss: 0.0226 - accuracy: 0.9913 - val_loss: 0.4335 - val_accuracy: 0.6989
Epoch 1/5
53/53 [==============================] - 11s 202ms/step - loss: 0.0158 - accuracy: 0.9953 - val_loss: 0.0113 - val_accuracy: 0.9959
Epoch 2/5
53/53 [==============================] - 11s 200ms/s

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 0.5557 - accuracy: 0.7639
Epoch 1/5
34/34 [==============================] - 7s 204ms/step - loss: 0.0224 - accuracy: 0.9910 - val_loss: 0.5868 - val_accuracy: 0.6099
Epoch 2/5
34/34 [==============================] - 7s 205ms/step - loss: 0.0205 - accuracy: 0.9918 - val_loss: 0.5391 - val_accuracy: 0.6417
Epoch 3/5
34/34 [==============================] - 7s 194ms/step - loss: 0.0200 - accuracy: 0.9922 - val_loss: 0.7830 - val_accuracy: 0.6062
Epoch 4/5
34/34 [==============================] - 7s 194ms/step - loss: 0.0197 - accuracy: 0.9921 - val_loss: 0.6072 - val_accuracy: 0.6515
Epoch 5/5
34/34 [==============================] - 7s 197ms/step - loss: 0.0192 - accuracy: 0.9925 - val_loss: 0.6149 - val_accuracy: 0.6597
Epoch 1/5
53/53 [==============================] - 11s 203ms/step - loss: 0.0151 - accuracy: 0.9955 - val_loss: 0.0100 - val_accuracy: 0.9970
Epoch 2/5
53/53 [==============================] - 11s 202ms/s

...

1721/1721 [==============================] - 8s 5ms/step - loss: 0.7098 - accuracy: 0.6930
Epoch 1/5
34/34 [==============================] - 7s 209ms/step - loss: 0.0193 - accuracy: 0.9922 - val_loss: 0.8127 - val_accuracy: 0.6313
Epoch 2/5
34/34 [==============================] - 7s 201ms/step - loss: 0.0180 - accuracy: 0.9927 - val_loss: 0.6380 - val_accuracy: 0.6730
Epoch 3/5
34/34 [==============================] - 7s 202ms/step - loss: 0.0176 - accuracy: 0.9929 - val_loss: 0.6726 - val_accuracy: 0.6698
Epoch 4/5
34/34 [==============================] - 7s 197ms/step - loss: 0.0174 - accuracy: 0.9930 - val_loss: 0.6316 - val_accuracy: 0.6844
Epoch 5/5
34/34 [==============================] - 7s 199ms/step - loss: 0.0175 - accuracy: 0.9930 - val_loss: 0.8891 - val_accuracy: 0.6277
Epoch 1/5
53/53 [==============================] - 11s 205ms/step - loss: 0.0130 - accuracy: 0.9960 - val_loss: 0.0088 - val_accuracy: 0.9968
Epoch 2/5
53/53 [==============================] - 11s 203ms/s

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 0.9803 - accuracy: 0.6435
Epoch 1/5
34/34 [==============================] - 7s 201ms/step - loss: 0.0166 - accuracy: 0.9932 - val_loss: 0.9286 - val_accuracy: 0.6299
Epoch 2/5
34/34 [==============================] - 7s 207ms/step - loss: 0.0160 - accuracy: 0.9933 - val_loss: 0.6608 - val_accuracy: 0.6848
Epoch 3/5
34/34 [==============================] - 7s 196ms/step - loss: 0.0155 - accuracy: 0.9933 - val_loss: 0.9764 - val_accuracy: 0.6188
Epoch 4/5
34/34 [==============================] - 7s 194ms/step - loss: 0.0154 - accuracy: 0.9937 - val_loss: 1.0115 - val_accuracy: 0.6231
Epoch 5/5
34/34 [==============================] - 7s 197ms/step - loss: 0.0155 - accuracy: 0.9936 - val_loss: 0.7279 - val_accuracy: 0.6853
Epoch 1/5
53/53 [==============================] - 11s 202ms/step - loss: 0.0113 - accuracy: 0.9966 - val_loss: 0.0102 - val_accuracy: 0.9958
Epoch 2/5
53/53 [==============================] - 10s 197ms/s

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 1.2249 - accuracy: 0.6604


In [144]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.17734451591968536, 0.3157421052455902], [0.26552197337150574, 0.5556977391242981], [0.22807778418064117, 0.7098347544670105], [0.2658305764198303, 0.9803367853164673], [0.24652689695358276, 1.224933385848999]]
Accuracy for iterations:  [[0.9863420128822327, 0.8860713243484497], [0.9422492384910583, 0.7639141082763672], [0.9682140946388245, 0.6929630041122437], [0.9415407776832581, 0.6435006856918335], [0.9373703598976135, 0.6604301333427429]]
F1 for iterations:  [[0.9863434071166878, 0.8834856798627919], [0.9419702803441444, 0.7429078457987311], [0.9681537449327864, 0.6456347353016861], [0.9411979544360722, 0.5664945427449044], [0.9369474222428824, 0.5947077139783268]]
Precision for iterations:  [[0.9863516318750827, 0.899635895551178], [0.945780408804803, 0.8245331394418728], [0.9691569229721725, 0.7916624194634834], [0.9461941562884257, 0.7730674081606458], [0.9429589206447899, 0.779775821415889]]
Recall for iterations:  [[0.9863419907096321, 0.8860713507229

#### DATASET 3G

In [145]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3G-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3G-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3G-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1708080388.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3G-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1708080388.py:3: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3G-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1708080388.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3G-Part3.csv')


In [146]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [147]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id906.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [148]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [149]:
global_model = build_model((24,1,1))

In [150]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [151]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id906.hdf5')

Epoch 1/5
45/45 [==============================] - 9s 210ms/step - loss: 0.2998 - accuracy: 0.9586 - val_loss: 2.1200 - val_accuracy: 0.5722
Epoch 2/5
45/45 [==============================] - 9s 210ms/step - loss: 0.0405 - accuracy: 0.9902 - val_loss: 0.6332 - val_accuracy: 0.6184
Epoch 3/5
45/45 [==============================] - 9s 199ms/step - loss: 0.0261 - accuracy: 0.9908 - val_loss: 0.4793 - val_accuracy: 0.7090
Epoch 4/5
45/45 [==============================] - 9s 202ms/step - loss: 0.0240 - accuracy: 0.9910 - val_loss: 0.4515 - val_accuracy: 0.7209
Epoch 5/5
45/45 [==============================] - 9s 199ms/step - loss: 0.0224 - accuracy: 0.9916 - val_loss: 0.4350 - val_accuracy: 0.7258
Epoch 1/5
65/65 [==============================] - 14s 211ms/step - loss: 0.0212 - accuracy: 0.9935 - val_loss: 0.5747 - val_accuracy: 0.6979
Epoch 2/5
65/65 [==============================] - 13s 204ms/step - loss: 0.0184 - accuracy: 0.9941 - val_loss: 0.3911 - val_accuracy: 0.7668
Epoch 3/5
6

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 10s 6ms/step - loss: 0.1897 - accuracy: 0.9159
Epoch 1/5
45/45 [==============================] - 10s 221ms/step - loss: 0.0159 - accuracy: 0.9949 - val_loss: 0.3127 - val_accuracy: 0.8173
Epoch 2/5
45/45 [==============================] - 10s 212ms/step - loss: 0.0147 - accuracy: 0.9953 - val_loss: 0.3413 - val_accuracy: 0.8140
Epoch 3/5
45/45 [==============================] - 9s 205ms/step - loss: 0.0144 - accuracy: 0.9956 - val_loss: 0.3549 - val_accuracy: 0.8148
Epoch 4/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0139 - accuracy: 0.9957 - val_loss: 0.3038 - val_accuracy: 0.8387
Epoch 5/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0135 - accuracy: 0.9959 - val_loss: 0.4867 - val_accuracy: 0.7711
Epoch 1/5
65/65 [==============================] - 14s 211ms/step - loss: 0.0141 - accuracy: 0.9962 - val_loss: 0.4849 - val_accuracy: 0.8032
Epoch 2/5
65/65 [==============================] - 13s 207m

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 4ms/step - loss: 0.5403 - accuracy: 0.7829
Epoch 1/5
45/45 [==============================] - 10s 212ms/step - loss: 0.0133 - accuracy: 0.9958 - val_loss: 0.3849 - val_accuracy: 0.8323
Epoch 2/5
45/45 [==============================] - 9s 208ms/step - loss: 0.0129 - accuracy: 0.9959 - val_loss: 0.6161 - val_accuracy: 0.7535
Epoch 3/5
45/45 [==============================] - 9s 204ms/step - loss: 0.0126 - accuracy: 0.9963 - val_loss: 0.4488 - val_accuracy: 0.8132
Epoch 4/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0125 - accuracy: 0.9961 - val_loss: 0.4488 - val_accuracy: 0.8162
Epoch 5/5
45/45 [==============================] - 9s 203ms/step - loss: 0.0126 - accuracy: 0.9961 - val_loss: 0.4100 - val_accuracy: 0.8202
Epoch 1/5
65/65 [==============================] - 14s 211ms/step - loss: 0.0125 - accuracy: 0.9962 - val_loss: 0.9648 - val_accuracy: 0.7417
Epoch 2/5
65/65 [==============================] - 13s 204ms/

...

1721/1721 [==============================] - 9s 5ms/step - loss: 0.8641 - accuracy: 0.6805
Epoch 1/5
45/45 [==============================] - 9s 211ms/step - loss: 0.0119 - accuracy: 0.9961 - val_loss: 0.6601 - val_accuracy: 0.7560
Epoch 2/5
45/45 [==============================] - 9s 206ms/step - loss: 0.0109 - accuracy: 0.9965 - val_loss: 0.5597 - val_accuracy: 0.7841
Epoch 3/5
45/45 [==============================] - 9s 202ms/step - loss: 0.0112 - accuracy: 0.9964 - val_loss: 0.5830 - val_accuracy: 0.7810
Epoch 4/5
45/45 [==============================] - 9s 211ms/step - loss: 0.0108 - accuracy: 0.9966 - val_loss: 0.6185 - val_accuracy: 0.7675
Epoch 5/5
45/45 [==============================] - 9s 205ms/step - loss: 0.0108 - accuracy: 0.9964 - val_loss: 0.6229 - val_accuracy: 0.7735
Epoch 1/5
65/65 [==============================] - 13s 203ms/step - loss: 0.0105 - accuracy: 0.9969 - val_loss: 0.4900 - val_accuracy: 0.8346
Epoch 2/5
65/65 [==============================] - 13s 201ms/s

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 10s 6ms/step - loss: 1.1946 - accuracy: 0.6446
Epoch 1/5
45/45 [==============================] - 9s 210ms/step - loss: 0.0103 - accuracy: 0.9967 - val_loss: 0.4169 - val_accuracy: 0.8258
Epoch 2/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0094 - accuracy: 0.9969 - val_loss: 0.5423 - val_accuracy: 0.7921
Epoch 3/5
45/45 [==============================] - 9s 205ms/step - loss: 0.0096 - accuracy: 0.9968 - val_loss: 0.4420 - val_accuracy: 0.8169
Epoch 4/5
45/45 [==============================] - 9s 201ms/step - loss: 0.0093 - accuracy: 0.9970 - val_loss: 0.5298 - val_accuracy: 0.8026
Epoch 5/5
45/45 [==============================] - 9s 204ms/step - loss: 0.0089 - accuracy: 0.9970 - val_loss: 0.7418 - val_accuracy: 0.7640
Epoch 1/5
65/65 [==============================] - 13s 204ms/step - loss: 0.0086 - accuracy: 0.9974 - val_loss: 0.6022 - val_accuracy: 0.8247
Epoch 2/5
65/65 [==============================] - 13s 201ms/

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 1.6098 - accuracy: 0.6298


In [152]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.0487123467028141, 0.189688578248024], [0.06193370372056961, 0.5402630567550659], [0.08920755237340927, 0.8641192317008972], [0.1142364889383316, 1.1945778131484985], [0.14707514643669128, 1.6098206043243408]]
Accuracy for iterations:  [[0.9868459701538086, 0.9159340262413025], [0.9707850217819214, 0.7829143404960632], [0.9662712812423706, 0.680483877658844], [0.9634593725204468, 0.644572377204895], [0.9591355323791504, 0.6298045516014099]]
F1 for iterations:  [[0.9868474118536541, 0.9148042428919093], [0.9707323040354474, 0.7665995668398837], [0.9661841271024689, 0.6274437679925693], [0.963345422776812, 0.568730634490003], [0.9589758880458612, 0.5423410530526002]]
Precision for iterations:  [[0.9868567431980539, 0.9227455525337379], [0.9716643857676753, 0.834378738223335], [0.9677966055439455, 0.7833643367329988], [0.9654693964573265, 0.7714825626667705], [0.9619029276240579, 0.7690532664079486]]
Recall for iterations:  [[0.986845949341202, 0.9159340260117707]

### Experimenting with epochs and iterations 

Use of dataset 3C, considering it contains an equal partition of the data between the 3 nodes. 

#### #Epochs = 10, #Iterations = 10

In [72]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low

In [73]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [74]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id910.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [75]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [76]:
global_model = build_model((24,1,1))

In [77]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [78]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id910.hdf5')

Epoch 1/10
54/54 [==============================] - 11s 199ms/step - loss: 0.2544 - accuracy: 0.9327 - val_loss: 1.8450 - val_accuracy: 0.6399
Epoch 2/10
54/54 [==============================] - 11s 197ms/step - loss: 0.0321 - accuracy: 0.9925 - val_loss: 0.7600 - val_accuracy: 0.6422
Epoch 3/10
54/54 [==============================] - 10s 192ms/step - loss: 0.0254 - accuracy: 0.9926 - val_loss: 0.5458 - val_accuracy: 0.6635
Epoch 4/10
54/54 [==============================] - 11s 196ms/step - loss: 0.0237 - accuracy: 0.9928 - val_loss: 0.4793 - val_accuracy: 0.6839
Epoch 5/10
54/54 [==============================] - 10s 194ms/step - loss: 0.0220 - accuracy: 0.9930 - val_loss: 0.5456 - val_accuracy: 0.6567
Epoch 6/10
54/54 [==============================] - 10s 193ms/step - loss: 0.0206 - accuracy: 0.9934 - val_loss: 0.4470 - val_accuracy: 0.7066
Epoch 7/10
54/54 [==============================] - 10s 192ms/step - loss: 0.0188 - accuracy: 0.9942 - val_loss: 0.5359 - val_accuracy: 0.6935

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 0.3768 - accuracy: 0.8759
Epoch 1/10
54/54 [==============================] - 11s 200ms/step - loss: 0.0137 - accuracy: 0.9961 - val_loss: 0.4771 - val_accuracy: 0.8012
Epoch 2/10
54/54 [==============================] - 11s 207ms/step - loss: 0.0130 - accuracy: 0.9960 - val_loss: 0.4833 - val_accuracy: 0.8056
Epoch 3/10
54/54 [==============================] - 11s 199ms/step - loss: 0.0127 - accuracy: 0.9963 - val_loss: 0.6188 - val_accuracy: 0.7750
Epoch 4/10
54/54 [==============================] - 11s 198ms/step - loss: 0.0127 - accuracy: 0.9963 - val_loss: 0.3453 - val_accuracy: 0.8471
Epoch 5/10
54/54 [==============================] - 11s 200ms/step - loss: 0.0125 - accuracy: 0.9961 - val_loss: 0.6800 - val_accuracy: 0.7706
Epoch 6/10
54/54 [==============================] - 11s 198ms/step - loss: 0.0122 - accuracy: 0.9965 - val_loss: 0.4442 - val_accuracy: 0.8306
Epoch 7/10
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 10s 6ms/step - loss: 1.0575 - accuracy: 0.6811 0s - loss: 1.0662 - accuracy: 0.
Epoch 1/10
54/54 [==============================] - 11s 196ms/step - loss: 0.0096 - accuracy: 0.9973 - val_loss: 0.5719 - val_accuracy: 0.8128
Epoch 2/10
54/54 [==============================] - 10s 193ms/step - loss: 0.0093 - accuracy: 0.9973 - val_loss: 0.5635 - val_accuracy: 0.8080
Epoch 3/10
54/54 [==============================] - 10s 191ms/step - loss: 0.0087 - accuracy: 0.9974 - val_loss: 0.6817 - val_accuracy: 0.7872
Epoch 4/10
54/54 [==============================] - 10s 188ms/step - loss: 0.0091 - accuracy: 0.9974 - val_loss: 0.3999 - val_accuracy: 0.8470
Epoch 5/10
54/54 [==============================] - 10s 191ms/step - loss: 0.0089 - accuracy: 0.9972 - val_loss: 0.5172 - val_accuracy: 0.8254
Epoch 6/10
54/54 [==============================] - 10s 190ms/step - loss: 0.0089 - accuracy: 0.9975 - val_loss: 0.5384 - val_accuracy: 0.8198
Epoch 7/10
54/54 

...

1721/1721 [==============================] - 8s 5ms/step - loss: 3.6558 - accuracy: 0.6425
Epoch 1/10
54/54 [==============================] - 11s 198ms/step - loss: 0.0056 - accuracy: 0.9985 - val_loss: 0.7006 - val_accuracy: 0.7890
Epoch 2/10
54/54 [==============================] - 10s 193ms/step - loss: 0.0052 - accuracy: 0.9984 - val_loss: 0.6449 - val_accuracy: 0.7970
Epoch 3/10
54/54 [==============================] - 10s 193ms/step - loss: 0.0052 - accuracy: 0.9984 - val_loss: 0.3634 - val_accuracy: 0.8604
Epoch 4/10
54/54 [==============================] - 10s 192ms/step - loss: 0.0054 - accuracy: 0.9984 - val_loss: 0.4947 - val_accuracy: 0.8304
Epoch 5/10
54/54 [==============================] - 10s 191ms/step - loss: 0.0049 - accuracy: 0.9985 - val_loss: 0.6775 - val_accuracy: 0.8014
Epoch 6/10
54/54 [==============================] - 10s 189ms/step - loss: 0.0051 - accuracy: 0.9986 - val_loss: 0.5711 - val_accuracy: 0.8132
Epoch 7/10
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 4.0920 - accuracy: 0.6729
Epoch 1/10
54/54 [==============================] - 11s 197ms/step - loss: 0.0047 - accuracy: 0.9985 - val_loss: 0.6507 - val_accuracy: 0.8058
Epoch 2/10
54/54 [==============================] - 11s 195ms/step - loss: 0.0046 - accuracy: 0.9986 - val_loss: 0.6399 - val_accuracy: 0.8050
Epoch 3/10
54/54 [==============================] - 11s 196ms/step - loss: 0.0045 - accuracy: 0.9985 - val_loss: 0.6329 - val_accuracy: 0.8100
Epoch 4/10
54/54 [==============================] - 11s 199ms/step - loss: 0.0045 - accuracy: 0.9986 - val_loss: 0.5471 - val_accuracy: 0.8214
Epoch 5/10
54/54 [==============================] - 10s 193ms/step - loss: 0.0049 - accuracy: 0.9983 - val_loss: 0.5311 - val_accuracy: 0.8320
Epoch 6/10
54/54 [==============================] - 10s 194ms/step - loss: 0.0044 - accuracy: 0.9986 - val_loss: 0.7088 - val_accuracy: 0.7913
Epoch 7/10
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 4.2869 - accuracy: 0.7279


In [79]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1511470228433609, 0.37683457136154175], [0.3587162494659424, 1.0574772357940674], [0.8660457134246826, 2.043614387512207], [1.6211916208267212, 2.6672563552856445], [2.25840163230896, 2.9753661155700684], [3.4451019763946533, 3.571256399154663], [3.8306243419647217, 3.4602181911468506], [4.477360248565674, 3.655755043029785], [5.541928291320801, 4.091959476470947], [6.338571071624756, 4.286871910095215]]
Accuracy for iterations:  [[0.9788848757743835, 0.8759173154830933], [0.7929971814155579, 0.6811015009880066], [0.5881343483924866, 0.5786892175674438], [0.5880978107452393, 0.5743842124938965], [0.5904861092567444, 0.5772542357444763], [0.5903181433677673, 0.5788345336914062], [0.5935537219047546, 0.6072622537612915], [0.5936924815177917, 0.6425379514694214], [0.5941160917282104, 0.6728910803794861], [0.593298077583313, 0.7279117703437805]]
F1 for iterations:  [[0.9788777632026339, 0.872614195868444], [0.7821209882007997, 0.6280498098219511], [0.4910611551908

#### #Epochs = 5, #Iterations = 5

In [5]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16700/1685572531.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16700/1685572531.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16700/1685572531.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16700/1685572531.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low

In [6]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [7]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id911.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [8]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [9]:
global_model = build_model((24,1,1))

In [10]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [11]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id911.hdf5')

Epoch 1/5
54/54 [==============================] - 12s 223ms/step - loss: 0.2645 - accuracy: 0.9722 - val_loss: 1.9327 - val_accuracy: 0.6399
Epoch 2/5
54/54 [==============================] - 12s 230ms/step - loss: 0.0342 - accuracy: 0.9921 - val_loss: 0.7014 - val_accuracy: 0.6454
Epoch 3/5
54/54 [==============================] - 13s 238ms/step - loss: 0.0256 - accuracy: 0.9925 - val_loss: 0.4979 - val_accuracy: 0.6941
Epoch 4/5
54/54 [==============================] - 12s 225ms/step - loss: 0.0239 - accuracy: 0.9925 - val_loss: 0.4756 - val_accuracy: 0.6957
Epoch 5/5
54/54 [==============================] - 12s 219ms/step - loss: 0.0224 - accuracy: 0.9927 - val_loss: 0.4330 - val_accuracy: 0.7127
Epoch 1/5
54/54 [==============================] - 12s 226ms/step - loss: 0.0200 - accuracy: 0.9934 - val_loss: 0.4135 - val_accuracy: 0.7392
Epoch 2/5
54/54 [==============================] - 12s 226ms/step - loss: 0.0171 - accuracy: 0.9946 - val_loss: 0.3682 - val_accuracy: 0.7957
Epoch 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 6s 4ms/step - loss: 0.2767 - accuracy: 0.8953
Epoch 1/5
54/54 [==============================] - 12s 220ms/step - loss: 0.0151 - accuracy: 0.9957 - val_loss: 0.3714 - val_accuracy: 0.8160
Epoch 2/5
54/54 [==============================] - 12s 217ms/step - loss: 0.0145 - accuracy: 0.9959 - val_loss: 0.4096 - val_accuracy: 0.8142
Epoch 3/5
54/54 [==============================] - 12s 221ms/step - loss: 0.0147 - accuracy: 0.9958 - val_loss: 0.4040 - val_accuracy: 0.8163
Epoch 4/5
54/54 [==============================] - 13s 238ms/step - loss: 0.0140 - accuracy: 0.9960 - val_loss: 0.4228 - val_accuracy: 0.8181
Epoch 5/5
54/54 [==============================] - 12s 231ms/step - loss: 0.0138 - accuracy: 0.9963 - val_loss: 0.3975 - val_accuracy: 0.8190
Epoch 1/5
54/54 [==============================] - 12s 219ms/step - loss: 0.0138 - accuracy: 0.9960 - val_loss: 0.5510 - val_accuracy: 0.7894
Epoch 2/5
54/54 [==============================] - 12s 22

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 0.6133 - accuracy: 0.7966
Epoch 1/5
54/54 [==============================] - 12s 228ms/step - loss: 0.0135 - accuracy: 0.9965 - val_loss: 0.4723 - val_accuracy: 0.8211
Epoch 2/5
54/54 [==============================] - 13s 234ms/step - loss: 0.0127 - accuracy: 0.9966 - val_loss: 0.4651 - val_accuracy: 0.8210
Epoch 3/5
54/54 [==============================] - 12s 225ms/step - loss: 0.0123 - accuracy: 0.9967 - val_loss: 0.6406 - val_accuracy: 0.7696
Epoch 4/5
54/54 [==============================] - 12s 219ms/step - loss: 0.0121 - accuracy: 0.9966 - val_loss: 0.5718 - val_accuracy: 0.7938
Epoch 5/5
54/54 [==============================] - 12s 226ms/step - loss: 0.0121 - accuracy: 0.9966 - val_loss: 0.5102 - val_accuracy: 0.8151
Epoch 1/5
54/54 [==============================] - 12s 228ms/step - loss: 0.0113 - accuracy: 0.9965 - val_loss: 0.4983 - val_accuracy: 0.8257
Epoch 2/5
54/54 [==============================] - 12s 22

...

1721/1721 [==============================] - 7s 4ms/step - loss: 0.8982 - accuracy: 0.7184
Epoch 1/5
54/54 [==============================] - 13s 235ms/step - loss: 0.0109 - accuracy: 0.9969 - val_loss: 0.3934 - val_accuracy: 0.8504
Epoch 2/5
54/54 [==============================] - 13s 240ms/step - loss: 0.0106 - accuracy: 0.9969 - val_loss: 0.7582 - val_accuracy: 0.7619
Epoch 3/5
54/54 [==============================] - 14s 252ms/step - loss: 0.0102 - accuracy: 0.9971 - val_loss: 0.6039 - val_accuracy: 0.8006
Epoch 4/5
54/54 [==============================] - 12s 228ms/step - loss: 0.0099 - accuracy: 0.9973 - val_loss: 0.3868 - val_accuracy: 0.8522
Epoch 5/5
54/54 [==============================] - 12s 221ms/step - loss: 0.0097 - accuracy: 0.9971 - val_loss: 0.5528 - val_accuracy: 0.8157
Epoch 1/5
54/54 [==============================] - 12s 224ms/step - loss: 0.0094 - accuracy: 0.9969 - val_loss: 0.6403 - val_accuracy: 0.8069
Epoch 2/5
54/54 [==============================] - 12s 22

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 6s 4ms/step - loss: 1.1148 - accuracy: 0.6839
Epoch 1/5
54/54 [==============================] - 12s 227ms/step - loss: 0.0093 - accuracy: 0.9974 - val_loss: 0.6650 - val_accuracy: 0.8055
Epoch 2/5
54/54 [==============================] - 12s 220ms/step - loss: 0.0095 - accuracy: 0.9976 - val_loss: 0.5259 - val_accuracy: 0.8258
Epoch 3/5
54/54 [==============================] - 12s 218ms/step - loss: 0.0084 - accuracy: 0.9977 - val_loss: 0.7564 - val_accuracy: 0.7706
Epoch 4/5
54/54 [==============================] - 12s 220ms/step - loss: 0.0081 - accuracy: 0.9978 - val_loss: 0.6562 - val_accuracy: 0.8006
Epoch 5/5
54/54 [==============================] - 12s 222ms/step - loss: 0.0084 - accuracy: 0.9977 - val_loss: 0.8334 - val_accuracy: 0.7509
Epoch 1/5
54/54 [==============================] - 15s 277ms/step - loss: 0.0083 - accuracy: 0.9973 - val_loss: 0.5401 - val_accuracy: 0.8260
Epoch 2/5
54/54 [==============================] - 12s 22

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 6s 4ms/step - loss: 1.2401 - accuracy: 0.6627


In [12]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.11703264713287354, 0.2767493426799774], [0.19591358304023743, 0.6133018732070923], [0.1857309639453888, 0.898158848285675], [0.18993905186653137, 1.1148115396499634], [0.2746482193470001, 1.2400809526443481]]
Accuracy for iterations:  [[0.9879488348960876, 0.8953171372413635], [0.9742615818977356, 0.7965922951698303], [0.9685500860214233, 0.7183572053909302], [0.9675859808921814, 0.6838988661766052], [0.8596073389053345, 0.6627007126808167]]
F1 for iterations:  [[0.9879512311722966, 0.893262750243958], [0.9742392353914436, 0.7831071675087792], [0.9684930401739139, 0.6826180191237142], [0.9675242307820903, 0.6323115245188365], [0.8560237382588983, 0.5989273184549058]]
Precision for iterations:  [[0.9879809146711573, 0.9066074075223175], [0.9745164002796086, 0.842080209017125], [0.9694262389720655, 0.8012362684054279], [0.9685306758294475, 0.786376489534689], [0.8846263110674074, 0.7781947755840517]]
Recall for iterations:  [[0.9879488153320284, 0.89531715469011

#### #Epochs = 5, #Iterations = 10

In [80]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10
# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [81]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id912.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [82]:
global_model = build_model((24,1,1))

In [83]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id912.hdf5')

Epoch 1/5


54/54 [==============================] - 11s 199ms/step - loss: 0.2511 - accuracy: 0.9069 - val_loss: 1.9361 - val_accuracy: 0.6399
Epoch 2/5
54/54 [==============================] - 11s 195ms/step - loss: 0.0319 - accuracy: 0.9925 - val_loss: 0.6612 - val_accuracy: 0.6454
Epoch 3/5
54/54 [==============================] - 10s 194ms/step - loss: 0.0251 - accuracy: 0.9926 - val_loss: 0.6359 - val_accuracy: 0.6476
Epoch 4/5
54/54 [==============================] - 10s 193ms/step - loss: 0.0235 - accuracy: 0.9927 - val_loss: 0.4588 - val_accuracy: 0.6863
Epoch 5/5
54/54 [==============================] - 10s 190ms/step - loss: 0.0220 - accuracy: 0.9928 - val_loss: 0.6081 - val_accuracy: 0.6489
Epoch 1/5
54/54 [==============================] - 10s 193ms/step - loss: 0.0200 - accuracy: 0.9934 - val_loss: 0.4392 - val_accuracy: 0.7208
Epoch 2/5
54/54 [==============================] - 11s 196ms/step - loss: 0.0170 - accuracy: 0.9945 - val_loss: 0.4354 - val_accuracy: 0.7638
Epoch 3/5
54/54 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 0.2919 - accuracy: 0.8903
Epoch 1/5
54/54 [==============================] - 10s 183ms/step - loss: 0.0151 - accuracy: 0.9955 - val_loss: 0.3542 - val_accuracy: 0.8211
Epoch 2/5
54/54 [==============================] - 10s 179ms/step - loss: 0.0146 - accuracy: 0.9958 - val_loss: 0.4645 - val_accuracy: 0.7932
Epoch 3/5
54/54 [==============================] - 9s 174ms/step - loss: 0.0142 - accuracy: 0.9959 - val_loss: 0.5226 - val_accuracy: 0.7890
Epoch 4/5
54/54 [==============================] - 10s 177ms/step - loss: 0.0140 - accuracy: 0.9961 - val_loss: 0.4800 - val_accuracy: 0.7964
Epoch 5/5
54/54 [==============================] - 10s 178ms/step - loss: 0.0140 - accuracy: 0.9960 - val_loss: 0.4435 - val_accuracy: 0.8102
Epoch 1/5
54/54 [==============================] - 10s 179ms/step - loss: 0.0133 - accuracy: 0.9960 - val_loss: 0.4236 - val_accuracy: 0.8256
Epoch 2/5
54/54 [==============================] - 10s 178

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


...

1721/1721 [==============================] - 9s 5ms/step - loss: 2.8040 - accuracy: 0.5842
Epoch 1/5
54/54 [==============================] - 10s 187ms/step - loss: 0.0073 - accuracy: 0.9981 - val_loss: 0.8659 - val_accuracy: 0.7804
Epoch 2/5
54/54 [==============================] - 10s 179ms/step - loss: 0.0068 - accuracy: 0.9981 - val_loss: 0.6685 - val_accuracy: 0.7993
Epoch 3/5
54/54 [==============================] - 10s 181ms/step - loss: 0.0069 - accuracy: 0.9981 - val_loss: 0.6449 - val_accuracy: 0.8058
Epoch 4/5
54/54 [==============================] - 10s 183ms/step - loss: 0.0070 - accuracy: 0.9982 - val_loss: 0.6463 - val_accuracy: 0.8061
Epoch 5/5
54/54 [==============================] - 10s 177ms/step - loss: 0.0066 - accuracy: 0.9983 - val_loss: 0.5415 - val_accuracy: 0.8284
Epoch 1/5
54/54 [==============================] - 10s 181ms/step - loss: 0.0070 - accuracy: 0.9978 - val_loss: 0.5380 - val_accuracy: 0.8321
Epoch 2/5
54/54 [==============================] - 10s 17

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 2.7545 - accuracy: 0.5972
Epoch 1/5
54/54 [==============================] - 11s 205ms/step - loss: 0.0067 - accuracy: 0.9982 - val_loss: 0.7035 - val_accuracy: 0.8026
Epoch 2/5
54/54 [==============================] - 10s 191ms/step - loss: 0.0070 - accuracy: 0.9980 - val_loss: 0.7450 - val_accuracy: 0.7860
Epoch 3/5
54/54 [==============================] - 10s 182ms/step - loss: 0.0067 - accuracy: 0.9982 - val_loss: 0.6474 - val_accuracy: 0.8085
Epoch 4/5
54/54 [==============================] - 10s 183ms/step - loss: 0.0065 - accuracy: 0.9983 - val_loss: 0.7878 - val_accuracy: 0.7786
Epoch 5/5
54/54 [==============================] - 10s 181ms/step - loss: 0.0064 - accuracy: 0.9983 - val_loss: 0.9783 - val_accuracy: 0.7380
Epoch 1/5
54/54 [==============================] - 10s 181ms/step - loss: 0.0068 - accuracy: 0.9979 - val_loss: 0.3654 - val_accuracy: 0.8654
Epoch 2/5
54/54 [==============================] - 10s 18

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 4ms/step - loss: 3.0288 - accuracy: 0.5879


In [84]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.09625285118818283, 0.2919221818447113], [0.12287917733192444, 0.5071086883544922], [0.1648828089237213, 0.8752776384353638], [0.11964885145425797, 1.0198652744293213], [0.29787641763687134, 1.54673171043396], [0.4365721344947815, 1.7502784729003906], [1.156044602394104, 2.4753761291503906], [1.8358707427978516, 2.8040149211883545], [1.7879897356033325, 2.75445294380188], [2.0196282863616943, 3.028756856918335]]
Accuracy for iterations:  [[0.9879561066627502, 0.8903218507766724], [0.9862908720970154, 0.821423351764679], [0.9732244610786438, 0.7205005884170532], [0.9810906052589417, 0.6993569731712341], [0.8586943745613098, 0.6328925490379333], [0.7638478875160217, 0.6436823606491089], [0.5811081528663635, 0.590823233127594], [0.5790995955467224, 0.5841749906539917], [0.584635853767395, 0.597199022769928], [0.5804653763771057, 0.5879350304603577]]
F1 for iterations:  [[0.9879585996628235, 0.8880029011684544], [0.9862914412058444, 0.8121348664985634], [0.97319277

#### #Epochs = 5, #Iterations = 15

In [85]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 15
# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [86]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id913.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [87]:
global_model = build_model((24,1,1))

In [88]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id913.hdf5')

Epoch 1/5


54/54 [==============================] - 10s 194ms/step - loss: 0.2479 - accuracy: 0.9556 - val_loss: 1.6721 - val_accuracy: 0.6400
Epoch 2/5
54/54 [==============================] - 10s 192ms/step - loss: 0.0310 - accuracy: 0.9924 - val_loss: 0.7515 - val_accuracy: 0.6422
Epoch 3/5
54/54 [==============================] - 10s 190ms/step - loss: 0.0249 - accuracy: 0.9925 - val_loss: 0.5051 - val_accuracy: 0.6767
Epoch 4/5
54/54 [==============================] - 10s 192ms/step - loss: 0.0232 - accuracy: 0.9925 - val_loss: 0.5618 - val_accuracy: 0.6519
Epoch 5/5
54/54 [==============================] - 10s 193ms/step - loss: 0.0217 - accuracy: 0.9927 - val_loss: 0.5475 - val_accuracy: 0.6590
Epoch 1/5
54/54 [==============================] - 10s 193ms/step - loss: 0.0200 - accuracy: 0.9933 - val_loss: 0.3913 - val_accuracy: 0.7403
Epoch 2/5
54/54 [==============================] - 10s 191ms/step - loss: 0.0173 - accuracy: 0.9944 - val_loss: 0.3801 - val_accuracy: 0.7717
Epoch 3/5
54/54 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 0.4205 - accuracy: 0.8769
Epoch 1/5
54/54 [==============================] - 11s 196ms/step - loss: 0.0151 - accuracy: 0.9957 - val_loss: 0.3374 - val_accuracy: 0.8307
Epoch 2/5
54/54 [==============================] - 10s 194ms/step - loss: 0.0145 - accuracy: 0.9959 - val_loss: 0.4684 - val_accuracy: 0.7910
Epoch 3/5
54/54 [==============================] - 11s 197ms/step - loss: 0.0144 - accuracy: 0.9958 - val_loss: 0.4533 - val_accuracy: 0.8001
Epoch 4/5
54/54 [==============================] - 10s 194ms/step - loss: 0.0142 - accuracy: 0.9960 - val_loss: 0.4413 - val_accuracy: 0.8081
Epoch 5/5
54/54 [==============================] - 11s 198ms/step - loss: 0.0135 - accuracy: 0.9963 - val_loss: 0.4525 - val_accuracy: 0.8089
Epoch 1/5
54/54 [==============================] - 11s 204ms/step - loss: 0.0131 - accuracy: 0.9962 - val_loss: 0.4575 - val_accuracy: 0.8284
Epoch 2/5
54/54 [==============================] - 11s 20

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


...

1721/1721 [==============================] - 8s 5ms/step - loss: 3.3669 - accuracy: 0.5978
Epoch 1/5
54/54 [==============================] - 12s 218ms/step - loss: 0.0067 - accuracy: 0.9983 - val_loss: 0.7341 - val_accuracy: 0.7993
Epoch 2/5
54/54 [==============================] - 12s 219ms/step - loss: 0.0059 - accuracy: 0.9983 - val_loss: 0.6709 - val_accuracy: 0.8084
Epoch 3/5
54/54 [==============================] - 12s 219ms/step - loss: 0.0058 - accuracy: 0.9984 - val_loss: 0.5303 - val_accuracy: 0.8337
Epoch 4/5
54/54 [==============================] - 12s 218ms/step - loss: 0.0063 - accuracy: 0.9982 - val_loss: 0.6994 - val_accuracy: 0.7977
Epoch 5/5
54/54 [==============================] - 12s 217ms/step - loss: 0.0058 - accuracy: 0.9984 - val_loss: 0.7557 - val_accuracy: 0.7952
Epoch 1/5
54/54 [==============================] - 12s 220ms/step - loss: 0.0064 - accuracy: 0.9981 - val_loss: 0.5858 - val_accuracy: 0.8311
Epoch 2/5
54/54 [==============================] - 12s 21

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 11s 6ms/step - loss: 3.4309 - accuracy: 0.6075
Epoch 1/5
54/54 [==============================] - 12s 222ms/step - loss: 0.0061 - accuracy: 0.9982 - val_loss: 0.4842 - val_accuracy: 0.8387
Epoch 2/5
54/54 [==============================] - 12s 222ms/step - loss: 0.0060 - accuracy: 0.9983 - val_loss: 0.7482 - val_accuracy: 0.7938
Epoch 3/5
54/54 [==============================] - 12s 220ms/step - loss: 0.0059 - accuracy: 0.9983 - val_loss: 0.7849 - val_accuracy: 0.7959
Epoch 4/5
54/54 [==============================] - 12s 220ms/step - loss: 0.0058 - accuracy: 0.9984 - val_loss: 0.6058 - val_accuracy: 0.8218
Epoch 5/5
54/54 [==============================] - 12s 229ms/step - loss: 0.0058 - accuracy: 0.9984 - val_loss: 0.5761 - val_accuracy: 0.8293
Epoch 1/5
54/54 [==============================] - 12s 222ms/step - loss: 0.0062 - accuracy: 0.9980 - val_loss: 0.6120 - val_accuracy: 0.8259
Epoch 2/5
54/54 [==============================] - 12s 2

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 3.5532 - accuracy: 0.6056


In [89]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.19311249256134033, 0.42053696513175964], [0.207519069314003, 0.7487212419509888], [0.2153296023607254, 1.1773408651351929], [0.20308400690555573, 1.4116877317428589], [0.39093777537345886, 1.8519067764282227], [0.584341287612915, 2.159454822540283], [0.8782985806465149, 2.4470252990722656], [1.0924036502838135, 2.6914265155792236], [1.4177628755569458, 2.7790586948394775], [1.517730474472046, 2.782214641571045], [1.855697512626648, 3.0099053382873535], [2.0155065059661865, 2.9566304683685303], [2.730656385421753, 3.3668792247772217], [2.6420443058013916, 3.4309136867523193], [2.999624252319336, 3.5531585216522217]]
Accuracy for iterations:  [[0.9832963347434998, 0.8768618702888489], [0.9705074429512024, 0.7520707845687866], [0.9358657598495483, 0.6496766805648804], [0.9405109882354736, 0.6331831812858582], [0.7832539677619934, 0.6055547595024109], [0.6764585375785828, 0.5994696021080017], [0.5852347612380981, 0.5861367583274841], [0.5860089659690857, 0.5898967

#### #Epochs = 10, #Iterations = 5

In [90]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5
# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [91]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id914.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [92]:
global_model = build_model((24,1,1))

In [93]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id914.hdf5')

Epoch 1/10


54/54 [==============================] - 12s 228ms/step - loss: 0.2533 - accuracy: 0.9727 - val_loss: 1.8887 - val_accuracy: 0.6399
Epoch 2/10
54/54 [==============================] - 12s 228ms/step - loss: 0.0327 - accuracy: 0.9924 - val_loss: 0.6309 - val_accuracy: 0.6492
Epoch 3/10
54/54 [==============================] - 12s 225ms/step - loss: 0.0257 - accuracy: 0.9926 - val_loss: 0.6105 - val_accuracy: 0.6527
Epoch 4/10
54/54 [==============================] - 12s 225ms/step - loss: 0.0239 - accuracy: 0.9926 - val_loss: 0.4485 - val_accuracy: 0.6978
Epoch 5/10
54/54 [==============================] - 12s 225ms/step - loss: 0.0226 - accuracy: 0.9928 - val_loss: 0.5086 - val_accuracy: 0.6659
Epoch 6/10
54/54 [==============================] - 12s 223ms/step - loss: 0.0208 - accuracy: 0.9932 - val_loss: 0.5316 - val_accuracy: 0.6664
Epoch 7/10
54/54 [==============================] - 12s 226ms/step - loss: 0.0193 - accuracy: 0.9939 - val_loss: 0.4171 - val_accuracy: 0.7278
Epoch 8/10

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 0.4283 - accuracy: 0.8469
Epoch 1/10
54/54 [==============================] - 12s 229ms/step - loss: 0.0140 - accuracy: 0.9961 - val_loss: 0.4353 - val_accuracy: 0.8186
Epoch 2/10
54/54 [==============================] - 12s 228ms/step - loss: 0.0132 - accuracy: 0.9962 - val_loss: 0.4390 - val_accuracy: 0.8173
Epoch 3/10
54/54 [==============================] - 12s 227ms/step - loss: 0.0134 - accuracy: 0.9963 - val_loss: 0.5241 - val_accuracy: 0.8004
Epoch 4/10
54/54 [==============================] - 12s 228ms/step - loss: 0.0129 - accuracy: 0.9965 - val_loss: 0.6577 - val_accuracy: 0.7691
Epoch 5/10
54/54 [==============================] - 12s 227ms/step - loss: 0.0129 - accuracy: 0.9965 - val_loss: 0.6586 - val_accuracy: 0.7787
Epoch 6/10
54/54 [==============================] - 12s 226ms/step - loss: 0.0128 - accuracy: 0.9964 - val_loss: 0.4950 - val_accuracy: 0.8198
Epoch 7/10
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


...

1721/1721 [==============================] - 8s 5ms/step - loss: 1.8330 - accuracy: 0.5882: 4s - loss: 4.2361 - accuracy - ETA - ETA: 3s - loss: 3.2569 - accura
Epoch 1/10
54/54 [==============================] - 13s 232ms/step - loss: 0.0080 - accuracy: 0.9977 - val_loss: 0.4665 - val_accuracy: 0.8357
Epoch 2/10
54/54 [==============================] - 14s 260ms/step - loss: 0.0089 - accuracy: 0.9973 - val_loss: 1.0048 - val_accuracy: 0.7306
Epoch 3/10
54/54 [==============================] - 13s 246ms/step - loss: 0.0081 - accuracy: 0.9978 - val_loss: 0.6301 - val_accuracy: 0.8093
Epoch 4/10
54/54 [==============================] - 13s 237ms/step - loss: 0.0074 - accuracy: 0.9981 - val_loss: 0.6060 - val_accuracy: 0.8070
Epoch 5/10
54/54 [==============================] - 13s 240ms/step - loss: 0.0073 - accuracy: 0.9979 - val_loss: 0.5361 - val_accuracy: 0.8231
Epoch 6/10
54/54 [==============================] - 13s 232ms/step - loss: 0.0078 - accuracy: 0.9979 - val_loss: 0.5998 - va

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 25s 14ms/step - loss: 2.4086 - accuracy: 0.5780
Epoch 1/10
54/54 [==============================] - 15s 282ms/step - loss: 0.0069 - accuracy: 0.9982 - val_loss: 0.7027 - val_accuracy: 0.8013
Epoch 2/10
54/54 [==============================] - 15s 281ms/step - loss: 0.0066 - accuracy: 0.9983 - val_loss: 0.8368 - val_accuracy: 0.7796
Epoch 3/10
54/54 [==============================] - 15s 279ms/step - loss: 0.0070 - accuracy: 0.9982 - val_loss: 0.7806 - val_accuracy: 0.7800
Epoch 4/10
54/54 [==============================] - 15s 284ms/step - loss: 0.0065 - accuracy: 0.9982 - val_loss: 0.4830 - val_accuracy: 0.8351
Epoch 5/10
54/54 [==============================] - 15s 272ms/step - loss: 0.0065 - accuracy: 0.9982 - val_loss: 0.5774 - val_accuracy: 0.8127
Epoch 6/10
54/54 [==============================] - 16s 294ms/step - loss: 0.0065 - accuracy: 0.9981 - val_loss: 0.5144 - val_accuracy: 0.8337
Epoch 7/10
54/54 [==============================]

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 22s 13ms/step - loss: 2.6894 - accuracy: 0.5911


In [94]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1365291327238083, 0.4283352196216583], [0.2651519179344177, 1.0024899244308472], [0.7037181258201599, 1.832952857017517], [1.5821514129638672, 2.4086146354675293], [2.3268370628356934, 2.6893749237060547]]
Accuracy for iterations:  [[0.9855824112892151, 0.8469265699386597], [0.860381543636322, 0.6725096106529236], [0.6239592432975769, 0.5882256627082825], [0.5767039656639099, 0.5780171751976013], [0.5847015976905823, 0.5910593867301941]]
F1 for iterations:  [[0.9855826670707051, 0.8409762834970507], [0.8568613387554346, 0.6149381434584613], [0.5508425246294328, 0.4636867477882422], [0.46738062441285394, 0.4423453790972255], [0.4837946867218398, 0.47302792034124125]]
Precision for iterations:  [[0.9855831374686987, 0.8727007844361674], [0.8850718746255002, 0.7805118410349414], [0.7680038756565108, 0.7443149963827191], [0.7453524292311242, 0.735034928827699], [0.7414682594253333, 0.7193566642547569]]
Recall for iterations:  [[0.9855824008881358, 0.84692654217830

#### #Epochs = 8, #Iterations = 5

In [95]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5
# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 8

In [96]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id915.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [97]:
global_model = build_model((24,1,1))

In [98]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id915.hdf5')

Epoch 1/8


54/54 [==============================] - 12s 228ms/step - loss: 0.2470 - accuracy: 0.9317 - val_loss: 1.6157 - val_accuracy: 0.6400
Epoch 2/8
54/54 [==============================] - 12s 222ms/step - loss: 0.0311 - accuracy: 0.9925 - val_loss: 0.7175 - val_accuracy: 0.6467
Epoch 3/8
54/54 [==============================] - 12s 219ms/step - loss: 0.0264 - accuracy: 0.9925 - val_loss: 0.6154 - val_accuracy: 0.6488
Epoch 4/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0246 - accuracy: 0.9926 - val_loss: 0.5837 - val_accuracy: 0.6496
Epoch 5/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0230 - accuracy: 0.9926 - val_loss: 0.4128 - val_accuracy: 0.7236
Epoch 6/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0211 - accuracy: 0.9933 - val_loss: 0.4062 - val_accuracy: 0.7333
Epoch 7/8
54/54 [==============================] - 12s 216ms/step - loss: 0.0192 - accuracy: 0.9939 - val_loss: 0.4178 - val_accuracy: 0.7333
Epoch 8/8
54/54 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 17s 10ms/step - loss: 0.3090 - accuracy: 0.9097
Epoch 1/8
54/54 [==============================] - 13s 232ms/step - loss: 0.0148 - accuracy: 0.9958 - val_loss: 0.5605 - val_accuracy: 0.7666
Epoch 2/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0140 - accuracy: 0.9960 - val_loss: 0.5251 - val_accuracy: 0.7822
Epoch 3/8
54/54 [==============================] - 12s 218ms/step - loss: 0.0142 - accuracy: 0.9959 - val_loss: 0.3481 - val_accuracy: 0.8477
Epoch 4/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0137 - accuracy: 0.9961 - val_loss: 0.6239 - val_accuracy: 0.7772
Epoch 5/8
54/54 [==============================] - 12s 215ms/step - loss: 0.0136 - accuracy: 0.9962 - val_loss: 0.4505 - val_accuracy: 0.8222
Epoch 6/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0132 - accuracy: 0.9962 - val_loss: 0.4862 - val_accuracy: 0.8098
Epoch 7/8
54/54 [==============================] - 12s 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


...

1721/1721 [==============================] - 16s 9ms/step - loss: 1.1401 - accuracy: 0.6861
Epoch 1/8
54/54 [==============================] - 12s 214ms/step - loss: 0.0084 - accuracy: 0.9978 - val_loss: 0.6121 - val_accuracy: 0.8078
Epoch 2/8
54/54 [==============================] - 12s 215ms/step - loss: 0.0085 - accuracy: 0.9977 - val_loss: 0.9500 - val_accuracy: 0.7377
Epoch 3/8
54/54 [==============================] - 12s 220ms/step - loss: 0.0079 - accuracy: 0.9978 - val_loss: 0.4002 - val_accuracy: 0.8463
Epoch 4/8
54/54 [==============================] - 13s 232ms/step - loss: 0.0077 - accuracy: 0.9978 - val_loss: 0.7166 - val_accuracy: 0.7872
Epoch 5/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0079 - accuracy: 0.9979 - val_loss: 0.6051 - val_accuracy: 0.8065
Epoch 6/8
54/54 [==============================] - 11s 203ms/step - loss: 0.0080 - accuracy: 0.9979 - val_loss: 0.5437 - val_accuracy: 0.8161
Epoch 7/8
54/54 [==============================] - 11s 2

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 6s 4ms/step - loss: 1.7186 - accuracy: 0.6793
Epoch 1/8
54/54 [==============================] - 11s 196ms/step - loss: 0.0072 - accuracy: 0.9981 - val_loss: 0.6565 - val_accuracy: 0.7961
Epoch 2/8
54/54 [==============================] - 11s 196ms/step - loss: 0.0071 - accuracy: 0.9981 - val_loss: 0.5742 - val_accuracy: 0.8095
Epoch 3/8
54/54 [==============================] - 11s 196ms/step - loss: 0.0067 - accuracy: 0.9982 - val_loss: 0.6854 - val_accuracy: 0.7932
Epoch 4/8
54/54 [==============================] - 10s 194ms/step - loss: 0.0067 - accuracy: 0.9980 - val_loss: 0.5630 - val_accuracy: 0.8158
Epoch 5/8
54/54 [==============================] - 11s 196ms/step - loss: 0.0070 - accuracy: 0.9982 - val_loss: 0.5873 - val_accuracy: 0.8081
Epoch 6/8
54/54 [==============================] - 11s 196ms/step - loss: 0.0067 - accuracy: 0.9982 - val_loss: 0.7150 - val_accuracy: 0.7843
Epoch 7/8
54/54 [==============================] - 10s 19

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 2.1888 - accuracy: 0.6872


In [99]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1551613062620163, 0.3090450167655945], [0.39441975951194763, 0.827255368232727], [0.48933789134025574, 1.1401209831237793], [1.3998279571533203, 1.718559980392456], [2.3852107524871826, 2.1888344287872314]]
Accuracy for iterations:  [[0.9876712560653687, 0.9097035527229309], [0.7235531210899353, 0.7248964905738831], [0.6320590972900391, 0.686133086681366], [0.5892736911773682, 0.6792668700218201], [0.5906614065170288, 0.6871684789657593]]
F1 for iterations:  [[0.9876740047741682, 0.9084097026691861], [0.6970280822167986, 0.6915926298900763], [0.5656337469397139, 0.6355295358394175], [0.4935263554174655, 0.6249145617594916], [0.49617727724775007, 0.6368594826811318]]
Precision for iterations:  [[0.987710573856323, 0.917019485897581], [0.8072991844827647, 0.8046626074435853], [0.7640255800302556, 0.788097391066705], [0.7370750903214642, 0.7857416156544874], [0.7372821739744763, 0.7895831169954107]]
Recall for iterations:  [[0.9876712728972509, 0.9097035530044322

#### #Epochs = 15, #Iterations = 5

In [100]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5
# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [101]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id916.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [102]:
global_model = build_model((24,1,1))

In [103]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id916.hdf5')

Epoch 1/15


54/54 [==============================] - 11s 207ms/step - loss: 0.2578 - accuracy: 0.9609 - val_loss: 1.8913 - val_accuracy: 0.6399
Epoch 2/15
54/54 [==============================] - 11s 195ms/step - loss: 0.0326 - accuracy: 0.9923 - val_loss: 0.6685 - val_accuracy: 0.6476
Epoch 3/15
54/54 [==============================] - 11s 201ms/step - loss: 0.0257 - accuracy: 0.9925 - val_loss: 0.5236 - val_accuracy: 0.6765
Epoch 4/15
54/54 [==============================] - 11s 211ms/step - loss: 0.0237 - accuracy: 0.9925 - val_loss: 0.4728 - val_accuracy: 0.6862
Epoch 5/15
54/54 [==============================] - 11s 209ms/step - loss: 0.0219 - accuracy: 0.9930 - val_loss: 0.4140 - val_accuracy: 0.7261
Epoch 6/15
54/54 [==============================] - 11s 206ms/step - loss: 0.0196 - accuracy: 0.9938 - val_loss: 0.4572 - val_accuracy: 0.7128
Epoch 7/15
54/54 [==============================] - 11s 210ms/step - loss: 0.0179 - accuracy: 0.9945 - val_loss: 0.3424 - val_accuracy: 0.7901
Epoch 8/15

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 0.6025 - accuracy: 0.8243
Epoch 1/15
54/54 [==============================] - 12s 218ms/step - loss: 0.0129 - accuracy: 0.9967 - val_loss: 0.5113 - val_accuracy: 0.8050
Epoch 2/15
54/54 [==============================] - 11s 210ms/step - loss: 0.0122 - accuracy: 0.9967 - val_loss: 0.4126 - val_accuracy: 0.8418
Epoch 3/15
54/54 [==============================] - 11s 208ms/step - loss: 0.0123 - accuracy: 0.9967 - val_loss: 0.4079 - val_accuracy: 0.8478
Epoch 4/15
54/54 [==============================] - 11s 212ms/step - loss: 0.0120 - accuracy: 0.9968 - val_loss: 0.4019 - val_accuracy: 0.8435
Epoch 5/15
54/54 [==============================] - 11s 210ms/step - loss: 0.0110 - accuracy: 0.9969 - val_loss: 0.5778 - val_accuracy: 0.7965
Epoch 6/15
54/54 [==============================] - 11s 212ms/step - loss: 0.0110 - accuracy: 0.9970 - val_loss: 0.4398 - val_accuracy: 0.8437
Epoch 7/15
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


...

1721/1721 [==============================] - 9s 5ms/step - loss: 2.0876 - accuracy: 0.6579
Epoch 1/15
54/54 [==============================] - 11s 205ms/step - loss: 0.0069 - accuracy: 0.9981 - val_loss: 0.4917 - val_accuracy: 0.8369
Epoch 2/15
54/54 [==============================] - 11s 201ms/step - loss: 0.0068 - accuracy: 0.9982 - val_loss: 0.5841 - val_accuracy: 0.8159
Epoch 3/15
54/54 [==============================] - 11s 197ms/step - loss: 0.0065 - accuracy: 0.9982 - val_loss: 0.7025 - val_accuracy: 0.8043
Epoch 4/15
54/54 [==============================] - 10s 194ms/step - loss: 0.0064 - accuracy: 0.9983 - val_loss: 0.6358 - val_accuracy: 0.8102
Epoch 5/15
54/54 [==============================] - 11s 195ms/step - loss: 0.0066 - accuracy: 0.9982 - val_loss: 0.8439 - val_accuracy: 0.7726
Epoch 6/15
54/54 [==============================] - 11s 196ms/step - loss: 0.0064 - accuracy: 0.9982 - val_loss: 0.9817 - val_accuracy: 0.7390
Epoch 7/15
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 10s 6ms/step - loss: 2.1810 - accuracy: 0.7010
Epoch 1/15
54/54 [==============================] - 11s 204ms/step - loss: 0.0064 - accuracy: 0.9982 - val_loss: 0.4371 - val_accuracy: 0.8478
Epoch 2/15
54/54 [==============================] - 11s 200ms/step - loss: 0.0062 - accuracy: 0.9984 - val_loss: 0.5495 - val_accuracy: 0.8274
Epoch 3/15
54/54 [==============================] - 11s 196ms/step - loss: 0.0058 - accuracy: 0.9983 - val_loss: 0.6802 - val_accuracy: 0.8021
Epoch 4/15
54/54 [==============================] - 11s 195ms/step - loss: 0.0061 - accuracy: 0.9984 - val_loss: 0.5591 - val_accuracy: 0.8186
Epoch 5/15
54/54 [==============================] - 10s 193ms/step - loss: 0.0055 - accuracy: 0.9985 - val_loss: 0.6556 - val_accuracy: 0.8105
Epoch 6/15
54/54 [==============================] - 11s 196ms/step - loss: 0.0078 - accuracy: 0.9982 - val_loss: 0.7606 - val_accuracy: 0.7853
Epoch 7/15
54/54 [==============================] 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 2.4714 - accuracy: 0.7287


In [104]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.21554751694202423, 0.6025245785713196], [0.43903639912605286, 1.4645129442214966], [1.2050604820251465, 2.087563991546631], [1.522641658782959, 2.1810131072998047], [2.0096006393432617, 2.4713964462280273]]
Accuracy for iterations:  [[0.951904833316803, 0.8243297338485718], [0.7666671276092529, 0.6635181307792664], [0.5929402112960815, 0.6579052805900574], [0.59264075756073, 0.7010281085968018], [0.5913698673248291, 0.7287473678588867]]
F1 for iterations:  [[0.9517577984522455, 0.815554872472297], [0.7510943322934576, 0.6004197509719447], [0.5016871463562266, 0.5910450327209422], [0.5031850823105745, 0.6578276036984575], [0.5035784985272762, 0.6969376973614473]]
Precision for iterations:  [[0.9537582253850795, 0.8581842500087389], [0.8281696470343505, 0.7777392125236178], [0.7319174911136448, 0.7766280385398969], [0.7224876012655619, 0.7937685972410671], [0.7104288177094473, 0.8061525739118192]]
Recall for iterations:  [[0.9519048175523679, 0.8243297246239919]

#### #Epochs = 5, #Iterations = 3

In [153]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low

In [154]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3
# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [155]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id917.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [156]:
global_model = build_model((24,1,1))

In [157]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id917.hdf5')

Epoch 1/5
45/45 [==============================] - 10s 213ms/step - loss: 0.2985 - accuracy: 0.9581 - val_loss: 2.1682 - val_accuracy: 0.5722
Epoch 2/5
45/45 [==============================] - 9s 210ms/step - loss: 0.0375 - accuracy: 0.9908 - val_loss: 0.6280 - val_accuracy: 0.6188
Epoch 3/5
45/45 [==============================] - 9s 209ms/step - loss: 0.0259 - accuracy: 0.9911 - val_loss: 0.5202 - val_accuracy: 0.6711
Epoch 4/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0238 - accuracy: 0.9914 - val_loss: 0.4142 - val_accuracy: 0.7493
Epoch 5/5
45/45 [==============================] - 9s 208ms/step - loss: 0.0219 - accuracy: 0.9921 - val_loss: 0.3805 - val_accuracy: 0.7698
Epoch 1/5
65/65 [==============================] - 14s 209ms/step - loss: 0.0209 - accuracy: 0.9936 - val_loss: 0.4311 - val_accuracy: 0.7335
Epoch 2/5
65/65 [==============================] - 13s 207ms/step - loss: 0.0176 - accuracy: 0.9946 - val_loss: 0.4078 - val_accuracy: 0.7782
Epoch 3/5


c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 0.2117 - accuracy: 0.9045
Epoch 1/5
45/45 [==============================] - 9s 206ms/step - loss: 0.0155 - accuracy: 0.9950 - val_loss: 0.4115 - val_accuracy: 0.7848
Epoch 2/5
45/45 [==============================] - 10s 212ms/step - loss: 0.0147 - accuracy: 0.9952 - val_loss: 0.3895 - val_accuracy: 0.8045
Epoch 3/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0145 - accuracy: 0.9956 - val_loss: 0.2868 - val_accuracy: 0.8383
Epoch 4/5
45/45 [==============================] - 9s 204ms/step - loss: 0.0140 - accuracy: 0.9958 - val_loss: 0.4138 - val_accuracy: 0.7975
Epoch 5/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0140 - accuracy: 0.9958 - val_loss: 0.3500 - val_accuracy: 0.8234
Epoch 1/5
65/65 [==============================] - 13s 206ms/step - loss: 0.0145 - accuracy: 0.9960 - val_loss: 0.3666 - val_accuracy: 0.8444
Epoch 2/5
65/65 [==============================] - 13s 204ms/

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 0.4155 - accuracy: 0.8119
Epoch 1/5
45/45 [==============================] - 9s 208ms/step - loss: 0.0136 - accuracy: 0.9957 - val_loss: 0.4478 - val_accuracy: 0.8064
Epoch 2/5
45/45 [==============================] - 9s 208ms/step - loss: 0.0131 - accuracy: 0.9959 - val_loss: 0.3384 - val_accuracy: 0.8421
Epoch 3/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0128 - accuracy: 0.9960 - val_loss: 0.4207 - val_accuracy: 0.8158
Epoch 4/5
45/45 [==============================] - 9s 203ms/step - loss: 0.0128 - accuracy: 0.9960 - val_loss: 0.3400 - val_accuracy: 0.8356
Epoch 5/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0127 - accuracy: 0.9961 - val_loss: 0.4374 - val_accuracy: 0.8109
Epoch 1/5
65/65 [==============================] - 13s 206ms/step - loss: 0.0125 - accuracy: 0.9965 - val_loss: 0.5329 - val_accuracy: 0.8134
Epoch 2/5
65/65 [==============================] - 13s 204ms/s

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 0.8198 - accuracy: 0.6863


In [158]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.055341463536024094, 0.21167635917663574], [0.054742418229579926, 0.4154618978500366], [0.09591921418905258, 0.8197669386863708]]
Accuracy for iterations:  [[0.984479546546936, 0.9045266509056091], [0.9725160002708435, 0.8118687868118286], [0.9642627835273743, 0.6862965822219849]]
F1 for iterations:  [[0.9844785269044698, 0.9029244478672587], [0.9724731963312149, 0.8011067238666677], [0.9641574953615748, 0.6362723102708954]]
Precision for iterations:  [[0.9844799221101318, 0.913737606198411], [0.9732125269378484, 0.8510417498580631], [0.9661124994480643, 0.7859974529417739]]
Recall for iterations:  [[0.9844795348973093, 0.904526629368597], [0.9725159952087411, 0.8118687786093148], [0.9642627596482515, 0.6862965923127226]]
